In [ ]:
#source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3])),1""""""""""""""""","""""""""""""""""blockIdx().y]",


# (1/sqrt( ((((shared_arr[(mod((mod((mod(threadIdx().x - 1, 5) + 1) - 1, 5) + 1) - 1, 5) + 1),1]-ceil( shared_arr[threadIdx().x,1]))*spacing[1])^2)",
# +(((shared_arr[threadIdx().x,2]-ceil( shared_arr[threadIdx().x,2]))*spacing[2])^2)",
# + (((shared_arr[threadIdx().x,3]-ceil( shared_arr[threadIdx().x,3]))*spacing[3])^2) )+0.000001",
# ))",

function get_interp_or_varr(is_var)
    res_vals=[]
    res_dists=[]
    # for x_s in ["ceil","floor"],y_s in ["ceil","floor"],z_s in ["ceil","floor"]
    for x_s in ["round(0.5 - 1e-10+","round((-0.5) + 1e-10+"],y_s in ["round(0.5 - 1e-10+","round((-0.5) + 1e-10+"],z_s in ["round(0.5 - 1e-10+","round((-0.5) + 1e-10+"]
        #in case it is variance we need to subtract the mean that is located in shared_arr[threadIdx().x,4]
        # dist_mono="""1"""
        dist_mono=""" (1/( ((((shared_arr[threadIdx().x,1]-$(x_s) shared_arr[threadIdx().x,1]))*spacing[1])^2)",
                        +(((shared_arr[threadIdx().x,2]-$(y_s) shared_arr[threadIdx().x,2]))*spacing[2])^2)",
                        + (((shared_arr[threadIdx().x,3]-$(z_s) shared_arr[threadIdx().x,3]))*spacing[3])^2) )+0.000001",
                        )) """

        if(is_var)
            strr=""" (((source_arr[  min(dims[1],max(1,Int($(x_s) shared_arr[threadIdx().x,1]))))",
            ,  min(dims[2],max(1,Int($(y_s) shared_arr[threadIdx().x,2]))))",
            ,  min(dims[3],max(1,Int($(z_s) shared_arr[threadIdx().x,3]))))",
            ,1,blockIdx().y] -shared_arr[threadIdx().x,4])^2)*($(dist_mono)))  """

        else
            strr=""" (source_arr[  min(dims[1],max(1,Int($(x_s) shared_arr[threadIdx().x,1]))))",
            ,  min(dims[2],max(1,Int($(y_s) shared_arr[threadIdx().x,2]))))",
            ,  min(dims[3],max(1,Int($(z_s) shared_arr[threadIdx().x,3]))))",
            ,1,blockIdx().y]*($(dist_mono)))  """
        end  

        push!(res_vals, strr)
        push!(res_dists, """($(dist_mono))""" )

    end
    val=join(res_vals, " + ")
    dist=join(res_dists, " + ")
    return """ (($(val))/($(dist)))
    
    """

end

# We need to first get min distance x  so we will know left or right min dist y so ant post and min dist z so inf sup 
# basically if else for each of the 3 dimensions should work
function get_nearest_neigh_interpol()
    res_vals=[]
    res_dists=[]
    for x_s in ["ceil","floor"],y_s in ["ceil","floor"],z_s in ["ceil","floor"]
        #in case it is variance we need to subtract the mean that is located in shared_arr[threadIdx().x,4]
        # dist_mono="""1"""
        dist_mono=""" (sqrt( ((((shared_arr[threadIdx().x,1]-$(x_s)( shared_arr[threadIdx().x,1]))*spacing[1])^2)",
                        +(((shared_arr[threadIdx().x,2]-$(y_s)( shared_arr[threadIdx().x,2]))*spacing[2])^2)",
                        + (((shared_arr[threadIdx().x,3]-$(z_s)( shared_arr[threadIdx().x,3]))*spacing[3])^2) )+0.000001",
                        )) """


        strr=""" (source_arr[Int($(x_s)( shared_arr[threadIdx().x,1])), Int($(y_s)( shared_arr[threadIdx().x,2])), Int($(z_s)( shared_arr[threadIdx().x,3])),1,blockIdx().y]*($(dist_mono)))  """


        push!(res_vals, strr)
        push!(res_dists, """($(dist_mono))""" )

    end
    val=join(res_vals, " + ")
    dist=join(res_dists, " + ")
    return """ (($(val))/($(dist)))
    
    """

end



In [ ]:
filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/debug.txt"
interp=get_interp_or_varr(false)
# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, """ $interp """ )
close(file)

In [ ]:
num_base_samp_points=2
num_additional_samp_points=1

interp=get_interp_or_varr(false)
varr=get_interp_or_varr(true)
# interp="""(((
#     source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1])))) +
#     source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1])))
#     )
#       *
#       (1 - (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2])))) +
#       (source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
#        +
#        source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(floor( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
#       *
#       (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2]))))
#      *
#      (1 - (shared_arr[threadIdx().x,3] - Int(floor( shared_arr[threadIdx().x,3]))))
#      +
#      ((source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
#        +
#        source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(floor( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
#       *
#       (1 - (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2])))) +
#       (source_arr[Int(floor( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (1 - (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
#        +
#        source_arr[Int(ceil( shared_arr[threadIdx().x,1])), Int(ceil( shared_arr[threadIdx().x,2])), Int(ceil( shared_arr[threadIdx().x,3])),1,blockIdx().y] * (shared_arr[threadIdx().x,1] - Int(floor( shared_arr[threadIdx().x,1]))))
#       *
#       (shared_arr[threadIdx().x,2] - Int(floor( shared_arr[threadIdx().x,2]))))
#      *
#      (shared_arr[threadIdx().x,3] - Int(floor( shared_arr[threadIdx().x,3]))))

# """

# varr="""(((
#     ((source_arr[Int(floor(shared_arr[threadIdx().x, 1])), Int(floor(shared_arr[threadIdx().x, 2])), Int(floor(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#     *
#     (1 - (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1])))) +
#     ((source_arr[Int(ceil(shared_arr[threadIdx().x, 1])), Int(floor(shared_arr[threadIdx().x, 2])), Int(floor(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#     *
#     (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1])))
#   )
#            *
#            (1 - (shared_arr[threadIdx().x, 2] - Int(floor(shared_arr[threadIdx().x, 2])))) +
#            (((source_arr[Int(floor(shared_arr[threadIdx().x, 1])), Int(ceil(shared_arr[threadIdx().x, 2])), Int(floor(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#             *
#             (1 - (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1]))))
#             +
#             ((source_arr[Int(ceil(shared_arr[threadIdx().x, 1])), Int(ceil(shared_arr[threadIdx().x, 2])), Int(floor(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#             *
#             (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1]))))
#            *
#            (shared_arr[threadIdx().x, 2] - Int(floor(shared_arr[threadIdx().x, 2]))))
#           *
#           (1 - (shared_arr[threadIdx().x, 3] - Int(floor(shared_arr[threadIdx().x, 3]))))
#           +
#           ((((source_arr[Int(floor(shared_arr[threadIdx().x, 1])), Int(floor(shared_arr[threadIdx().x, 2])), Int(ceil(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#             *
#             (1 - (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1]))))
#             +
#             ((source_arr[Int(ceil(shared_arr[threadIdx().x, 1])), Int(floor(shared_arr[threadIdx().x, 2])), Int(ceil(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#             *
#             (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1]))))
#            *
#            (1 - (shared_arr[threadIdx().x, 2] - Int(floor(shared_arr[threadIdx().x, 2])))) +
#            (((source_arr[Int(floor(shared_arr[threadIdx().x, 1])), Int(ceil(shared_arr[threadIdx().x, 2])), Int(ceil(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#             *
#             (1 - (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1]))))
#             +
#             ((source_arr[Int(ceil(shared_arr[threadIdx().x, 1])), Int(ceil(shared_arr[threadIdx().x, 2])), Int(ceil(shared_arr[threadIdx().x, 3])),1,blockIdx().y] - shared_arr[threadIdx().x, 4])^2)
#             *
#             (shared_arr[threadIdx().x, 1] - Int(floor(shared_arr[threadIdx().x, 1]))))
#            *
#            (shared_arr[threadIdx().x, 2] - Int(floor(shared_arr[threadIdx().x, 2]))))
#           *
#           (shared_arr[threadIdx().x, 3] - Int(floor(shared_arr[threadIdx().x, 3]))))

# """


res=""
for point_num in 1:num_base_samp_points
res=res*"""
#we get the diffrence between the sv center and the triangle center
      shared_arr[threadIdx().x,1]= ((tetr_dat[index,5,1,blockIdx().y]-tetr_dat[index,1,1,blockIdx().y])/($num_base_samp_points+1))
      shared_arr[threadIdx().x,2]=((tetr_dat[index,5,2,blockIdx().y]-tetr_dat[index,1,2,blockIdx().y])/($num_base_samp_points+1))
      shared_arr[threadIdx().x,3]=((tetr_dat[index,5,3,blockIdx().y]-tetr_dat[index,1,3,blockIdx().y])/($num_base_samp_points+1))



      ##calculate weight of the point
      #first distance from next and previous point on the line between sv center and triangle center
      shared_arr[threadIdx().x,4]=sqrt((shared_arr[threadIdx().x,1])^2 +(shared_arr[threadIdx().x,2])^2+(shared_arr[threadIdx().x,3])^2)*2 #distance between main sample points (two times for distance to previous and next)
      #now we get the distance to the lines that get from sv center to the triangle corners - for simplicity
      # we can assume that sv center location is 0.0,0.0,0.0 as we need only diffrences 
      #now we get the location of sample point

      shared_arr[threadIdx().x,1]= tetr_dat[index,1,1,blockIdx().y]+(shared_arr[threadIdx().x,1]*$point_num)
      shared_arr[threadIdx().x,2]= tetr_dat[index,1,2,blockIdx().y]+(shared_arr[threadIdx().x,2]*$point_num)
      shared_arr[threadIdx().x,3]= tetr_dat[index,1,3,blockIdx().y]+(shared_arr[threadIdx().x,3]*$point_num)

"""
    for triangle_corner_num in 1:3
        if(point_num==num_base_samp_points)
            res=res*"""

            shared_arr[threadIdx().x,4]+=sqrt( ((tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]-shared_arr[threadIdx().x,1])
                  *(1/(num_additional_samp_points+1)))^2
                  +((tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]-shared_arr[threadIdx().x,2])
                  *(1/(num_additional_samp_points+1)))^2
                  +((tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]-shared_arr[threadIdx().x,3])
                  *(1/(num_additional_samp_points+1)))^2)

                   """
        else
            res=res*"""

    shared_arr[threadIdx().x,4]+=(sqrt(((((tetr_dat[index,1,2,blockIdx().y]
    -tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y])
    *(tetr_dat[index,1,3,blockIdx().y]-shared_arr[threadIdx().x,3]) 
    - (tetr_dat[index,1,3,blockIdx().y]-tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y])
    *(tetr_dat[index,1,2,blockIdx().y]-shared_arr[threadIdx().x,2]))^2)+
              (((tetr_dat[index,1,3,blockIdx().y]
              -tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y])
              *(tetr_dat[index,1,1,blockIdx().y]-shared_arr[threadIdx().x,1])
               - (tetr_dat[index,1,1,blockIdx().y]-tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y])
               *(tetr_dat[index,1,3,blockIdx().y]-shared_arr[threadIdx().x,3]))^2)+
              ((tetr_dat[index,1,1,blockIdx().y]-tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y])
              *(tetr_dat[index,1,2,blockIdx().y]-shared_arr[threadIdx().x,2]) 
              - (tetr_dat[index,1,2,blockIdx().y]-tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y])
              *(tetr_dat[index,1,1,blockIdx().y]-shared_arr[threadIdx().x,1]))^2)) 
              / (sqrt((tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]
              -tetr_dat[index,1,2,blockIdx().y])^2
              +(tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]
              -tetr_dat[index,1,3,blockIdx().y])^2
              +(tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]
              -tetr_dat[index,1,1,blockIdx().y])^2)+0.00001)  )
              
              out_sampled_points[index,$point_num,1,blockIdx().y]=$interp
              #saving sample points coordinates mainly for debugging and visualizations
              out_sampled_points[index,$point_num,3,blockIdx().y]=shared_arr[threadIdx().x,1]
              out_sampled_points[index,$point_num,4,blockIdx().y]=shared_arr[threadIdx().x,2]
              out_sampled_points[index,$point_num,5,blockIdx().y]=shared_arr[threadIdx().x,3]

            """            
        end    

    end#triangle_corner_num
    res=res*""" out_sampled_points[index,$point_num,2,blockIdx().y]= (((shared_arr[threadIdx().x,4])/5)^3)
    out_sampled_points[index,$point_num,1,blockIdx().y]=$interp
    #saving sample points coordinates mainly for debugging and visualizations
    out_sampled_points[index,$point_num,3,blockIdx().y]=shared_arr[threadIdx().x,1]
    out_sampled_points[index,$point_num,4,blockIdx().y]=shared_arr[threadIdx().x,2]
    out_sampled_points[index,$point_num,5,blockIdx().y]=shared_arr[threadIdx().x,3]
    
    
    """

end


for n_add_samp in UInt8(1):UInt8(num_additional_samp_points)
    for triangle_corner_num in UInt8(1):UInt8(3)
        res=res*""" 
        shared_arr[threadIdx().x,1]=((tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]-out_sampled_points[index,num_base_samp_points,3,blockIdx().y])/($num_additional_samp_points+1))
        shared_arr[threadIdx().x,2]=((tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]-out_sampled_points[index,num_base_samp_points,4,blockIdx().y])/($num_additional_samp_points+1))
        shared_arr[threadIdx().x,3]=((tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]-out_sampled_points[index,num_base_samp_points,5,blockIdx().y])/($num_additional_samp_points+1))

        #### calculate weight of the point 
        #first we get distance to the previous and next point on the line between the last main sample point and the triangle corner
        shared_arr[threadIdx().x,4]=sqrt( shared_arr[threadIdx().x,1]^2+shared_arr[threadIdx().x,2]^2+shared_arr[threadIdx().x,3]^2)*2
        #now we get the location of sample point
        shared_arr[threadIdx().x,1]= out_sampled_points[index,num_base_samp_points,3,blockIdx().y]+(shared_arr[threadIdx().x,1]*($n_add_samp))
        shared_arr[threadIdx().x,2]= out_sampled_points[index,num_base_samp_points,4,blockIdx().y]+(shared_arr[threadIdx().x,2]*($n_add_samp))
        shared_arr[threadIdx().x,3]= out_sampled_points[index,num_base_samp_points,5,blockIdx().y]+(shared_arr[threadIdx().x,3]*($n_add_samp))

        """
        for tetr_dat_index in UInt8(1):UInt8(4)
            if(tetr_dat_index!=(triangle_corner_num+1) )
                res=res*""" 

               shared_arr[threadIdx().x,4]+=(sqrt(((((tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]
               -tetr_dat[index,$tetr_dat_index,2,blockIdx().y])
               *(tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]-shared_arr[threadIdx().x,3]) 
               - (tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]
               -tetr_dat[index,$tetr_dat_index,3,blockIdx().y])
               *(tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]-shared_arr[threadIdx().x,2]))^2)+
                (((tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]
                -tetr_dat[index,$tetr_dat_index,3,blockIdx().y])
                *(tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]-shared_arr[threadIdx().x,1]) 
                - (tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]
                -tetr_dat[index,$tetr_dat_index,1,blockIdx().y])
                *(tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]-shared_arr[threadIdx().x,3]))^2)+
                ((tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]
                -tetr_dat[index,$tetr_dat_index,1,blockIdx().y])
                *(tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]-shared_arr[threadIdx().x,2]) 
                - (tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]
                -tetr_dat[index,$tetr_dat_index,2,blockIdx().y])
                *(tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]-shared_arr[threadIdx().x,1]))^2)) 
                / (sqrt((tetr_dat[index,$tetr_dat_index,2,blockIdx().y]
                -tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y])^2
                +(tetr_dat[index,$tetr_dat_index,3,blockIdx().y]
                -tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y])^2
                +(tetr_dat[index,$tetr_dat_index,1,blockIdx().y]
                -tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y])^2) +0.00000001) )

                
                """
            end#if
        end#tetr_dat_index    
        res=res*""" 
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,2,blockIdx().y]=(((shared_arr[threadIdx().x,4])/5)^3)      
        #performing interpolation result is in var2 and it get data from shared_arr
        #saving the result of interpolated value to the out_sampled_points
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,1,blockIdx().y]=$interp
        # #saving sample points coordinates
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,3,blockIdx().y]=shared_arr[threadIdx().x,1]
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,4,blockIdx().y]=shared_arr[threadIdx().x,2]
        out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,5,blockIdx().y]=shared_arr[threadIdx().x,3]
        
        """


    end#triangle_corner_num
    
end#n_add_samp    
# Specify the file path
filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/unrolled.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


In [ ]:
interp

In [ ]:
# res=""
# for point_num in 1:num_base_samp_points
# res=res*"""
# #we get the diffrence between the sv center and the triangle center
#       shared_arr[threadIdx().x,1]= ((tetr_dat[index,5,1,blockIdx().y]-tetr_dat[index,1,1,blockIdx().y])/($num_base_samp_points+1))
#       shared_arr[threadIdx().x,2]=((tetr_dat[index,5,2,blockIdx().y]-tetr_dat[index,1,2,blockIdx().y])/($num_base_samp_points+1))
#       shared_arr[threadIdx().x,3]=((tetr_dat[index,5,3,blockIdx().y]-tetr_dat[index,1,3,blockIdx().y])/($num_base_samp_points+1))



#       ##calculate weight of the point
#       #first distance from next and previous point on the line between sv center and triangle center
#       shared_arr[threadIdx().x,4]=sqrt((shared_arr[threadIdx().x,1])^2 +(shared_arr[threadIdx().x,2])^2+(shared_arr[threadIdx().x,3])^2)*2 #distance between main sample points (two times for distance to previous and next)
#       #now we get the distance to the lines that get from sv center to the triangle corners - for simplicity
#       # we can assume that sv center location is 0.0,0.0,0.0 as we need only diffrences 
#       #now we get the location of sample point

#       shared_arr[threadIdx().x,1]= tetr_dat[index,1,1,blockIdx().y]+(shared_arr[threadIdx().x,1]*$point_num)
#       shared_arr[threadIdx().x,2]= tetr_dat[index,1,2,blockIdx().y]+(shared_arr[threadIdx().x,2]*$point_num)
#       shared_arr[threadIdx().x,3]= tetr_dat[index,1,3,blockIdx().y]+(shared_arr[threadIdx().x,3]*$point_num)

# """
#     for triangle_corner_num in 1:3
#         if(point_num==num_base_samp_points)
#             res=res*"""

#             shared_arr[threadIdx().x,4]+=sqrt( ((tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]-shared_arr[threadIdx().x,1])
#                   *(1/(num_additional_samp_points+1)))^2
#                   +((tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]-shared_arr[threadIdx().x,2])
#                   *(1/(num_additional_samp_points+1)))^2
#                   +((tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]-shared_arr[threadIdx().x,3])
#                   *(1/(num_additional_samp_points+1)))^2)

#                    """
#         else
#             res=res*"""

#     shared_arr[threadIdx().x,4]+=(sqrt(((((tetr_dat[index,1,2]-tetr_dat[index,$triangle_corner_num+1,2])*(tetr_dat[index,1,3]-shared_arr[threadIdx().x,3]) - (tetr_dat[index,1,3]-tetr_dat[index,$triangle_corner_num+1,3])*(tetr_dat[index,1,2]-shared_arr[threadIdx().x,2]))^2)+
#               (((tetr_dat[index,1,3]-tetr_dat[index,$triangle_corner_num+1,3])*(tetr_dat[index,1,1]-shared_arr[threadIdx().x,1]) - (tetr_dat[index,1,1]-tetr_dat[index,$triangle_corner_num+1,1])*(tetr_dat[index,1,3]-shared_arr[threadIdx().x,3]))^2)+
#               ((tetr_dat[index,1,1]-tetr_dat[index,$triangle_corner_num+1,1])*(tetr_dat[index,1,2]-shared_arr[threadIdx().x,2]) - (tetr_dat[index,1,2]-tetr_dat[index,$triangle_corner_num+1,2])*(tetr_dat[index,1,1]-shared_arr[threadIdx().x,1]))^2)) 
#               / (sqrt((tetr_dat[index,$triangle_corner_num+1,2]-tetr_dat[index,1,2])^2+(tetr_dat[index,$triangle_corner_num+1,3]-tetr_dat[index,1,3])^2+(tetr_dat[index,$triangle_corner_num+1,1]-tetr_dat[index,1,1])^2)+0.00001)  )
              
#               out_sampled_points[index,$point_num,1,blockIdx().y]=$interp
#               #saving sample points coordinates mainly for debugging and visualizations
#               out_sampled_points[index,$point_num,3,blockIdx().y]=shared_arr[threadIdx().x,1]
#               out_sampled_points[index,$point_num,4,blockIdx().y]=shared_arr[threadIdx().x,2]
#               out_sampled_points[index,$point_num,5,blockIdx().y]=shared_arr[threadIdx().x,3]

#             """            
#         end    

#     end#triangle_corner_num
#     res=res*""" out_sampled_points[index,$point_num,2,blockIdx().y]= (((shared_arr[threadIdx().x,4])/5)^3)
#     out_sampled_points[index,$point_num,1,blockIdx().y]=$interp
#     #saving sample points coordinates mainly for debugging and visualizations
#     out_sampled_points[index,$point_num,3,blockIdx().y]=shared_arr[threadIdx().x,1]
#     out_sampled_points[index,$point_num,4,blockIdx().y]=shared_arr[threadIdx().x,2]
#     out_sampled_points[index,$point_num,5,blockIdx().y]=shared_arr[threadIdx().x,3]
    
    
#     """

# end


# for n_add_samp in UInt8(1):UInt8(num_additional_samp_points)
#     for triangle_corner_num in UInt8(1):UInt8(3)
#         res=res*""" 
#         shared_arr[threadIdx().x,1]=((tetr_dat[index,$triangle_corner_num+1,1,blockIdx().y]-out_sampled_points[index,num_base_samp_points,3,blockIdx().y])/($num_additional_samp_points+1))
#         shared_arr[threadIdx().x,2]=((tetr_dat[index,$triangle_corner_num+1,2,blockIdx().y]-out_sampled_points[index,num_base_samp_points,4,blockIdx().y])/($num_additional_samp_points+1))
#         shared_arr[threadIdx().x,3]=((tetr_dat[index,$triangle_corner_num+1,3,blockIdx().y]-out_sampled_points[index,num_base_samp_points,5,blockIdx().y])/($num_additional_samp_points+1))

#         #### calculate weight of the point 
#         #first we get distance to the previous and next point on the line between the last main sample point and the triangle corner
#         shared_arr[threadIdx().x,4]=sqrt( shared_arr[threadIdx().x,1]^2+shared_arr[threadIdx().x,2]^2+shared_arr[threadIdx().x,3]^2)*2
#         #now we get the location of sample point
#         shared_arr[threadIdx().x,1]= out_sampled_points[index,num_base_samp_points,3,blockIdx().y]+(shared_arr[threadIdx().x,1]*($n_add_samp))
#         shared_arr[threadIdx().x,2]= out_sampled_points[index,num_base_samp_points,4,blockIdx().y]+(shared_arr[threadIdx().x,2]*($n_add_samp))
#         shared_arr[threadIdx().x,3]= out_sampled_points[index,num_base_samp_points,5,blockIdx().y]+(shared_arr[threadIdx().x,3]*($n_add_samp))

#         """
#         for tetr_dat_index in UInt8(1):UInt8(4)
#             if(tetr_dat_index!=(triangle_corner_num+1) )
#                 res=res*""" 

#                shared_arr[threadIdx().x,4]+=(sqrt(((((tetr_dat[index,$triangle_corner_num+1,2]-tetr_dat[index,$tetr_dat_index,2])*(tetr_dat[index,$triangle_corner_num+1,3]-shared_arr[threadIdx().x,3]) - (tetr_dat[index,$triangle_corner_num+1,3]-tetr_dat[index,$tetr_dat_index,3])*(tetr_dat[index,$triangle_corner_num+1,2]-shared_arr[threadIdx().x,2]))^2)+
#                 (((tetr_dat[index,$triangle_corner_num+1,3]-tetr_dat[index,$tetr_dat_index,3])*(tetr_dat[index,$triangle_corner_num+1,1]-shared_arr[threadIdx().x,1]) - (tetr_dat[index,$triangle_corner_num+1,1]-tetr_dat[index,$tetr_dat_index,1])*(tetr_dat[index,$triangle_corner_num+1,3]-shared_arr[threadIdx().x,3]))^2)+
#                 ((tetr_dat[index,$triangle_corner_num+1,1]-tetr_dat[index,$tetr_dat_index,1])*(tetr_dat[index,$triangle_corner_num+1,2]-shared_arr[threadIdx().x,2]) - (tetr_dat[index,$triangle_corner_num+1,2]-tetr_dat[index,$tetr_dat_index,2])*(tetr_dat[index,$triangle_corner_num+1,1]-shared_arr[threadIdx().x,1]))^2)) 
#                 / (sqrt((tetr_dat[index,$tetr_dat_index,2]-tetr_dat[index,$triangle_corner_num+1,2])^2+(tetr_dat[index,$tetr_dat_index,3]-tetr_dat[index,$triangle_corner_num+1,3])^2+(tetr_dat[index,$tetr_dat_index,1]-tetr_dat[index,$triangle_corner_num+1,1])^2) +0.00000001) )

                
#                 """
#             end#if
#         end#tetr_dat_index    
#         res=res*""" 
#         out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,2,blockIdx().y]=(((shared_arr[threadIdx().x,4])/5)^3)      
#         #performing interpolation result is in var2 and it get data from shared_arr
#         #saving the result of interpolated value to the out_sampled_points
#         out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,1,blockIdx().y]=$interp
#         # #saving sample points coordinates
#         out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,3,blockIdx().y]=shared_arr[threadIdx().x,1]
#         out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,4,blockIdx().y]=shared_arr[threadIdx().x,2]
#         out_sampled_points[index,(num_base_samp_points+$triangle_corner_num)+($n_add_samp-1)*3,5,blockIdx().y]=shared_arr[threadIdx().x,3]
        
#         """


#     end#triangle_corner_num
    
# end#n_add_samp    

In [ ]:
a=" r "
b= " u "
a*b

# set_tetr_dat_kern_forward

In [ ]:
res=""

for i in 1:3
res=res*"""  shared_arr[threadIdx().x, $(i)] = sv_centers[Int(tetr_dat[index, 1, 1]), Int(tetr_dat[index, 1, 2]), Int(tetr_dat[index, 1, 3]), $(i),blockIdx().y]
 """
end
for i in 1:3
res=res*"""  tetr_dat_out[index, 1, $(i),blockIdx().y] = shared_arr[threadIdx().x, $(i)]
 """
end
res=res*"""  tetr_dat_out[index, 1, 4,blockIdx().y] = $(interp)
 """
for triangle_corner_num in UInt8(2):UInt8(4)

    for i in 1:3
        res=res*"""  shared_arr[threadIdx().x, $(i)] = control_points[Int(tetr_dat[index, $triangle_corner_num, 1])
        , Int(tetr_dat[index, $triangle_corner_num, 2])
        , Int(tetr_dat[index, $triangle_corner_num, 3])
        , Int(tetr_dat[index, $triangle_corner_num, 4]), $(i)]
         """
        end

    for i in 1:3
        res=res*"""  tetr_dat_out[index, $triangle_corner_num, $(i),blockIdx().y] = shared_arr[threadIdx().x, $(i)]
            """
        end

    res=res*"""
    shared_arr[threadIdx().x, 4] = $(interp)
    tetr_dat_out[index, $triangle_corner_num, 4,blockIdx().y] = $(varr) """

    
end    


for triangle_corner_num in UInt8(2):UInt8(3)
    for i in 1:3
        res=res*"""  shared_arr[threadIdx().x, $(i)] += tetr_dat_out[index, $triangle_corner_num, $(i),blockIdx().y]
            """
        end
    end    


        for axis in UInt8(1):UInt8(3)
        res=res*"""      shared_arr[threadIdx().x, $axis] = shared_arr[threadIdx().x, $axis] / 3
    tetr_dat_out[index, 5, $axis,blockIdx().y] = shared_arr[threadIdx().x, $axis]
            """
        end
res=res*"""
shared_arr[threadIdx().x, 4] = $(interp)
tetr_dat_out[index, 5, 4,blockIdx().y] = $(varr) """

filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/unrolled_tetr_dat_kern.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


# now we need to 
# 1) print the code from above and use ctrl f step by step to check is it identical to already present unrolled Version
# 2) add support for batches - basically add blockid.y to each array at the end
# 3) add support for channels here we basically need separate unctions - kernels so we will simply sample variance  from tetr_dat or iterpolated value from out_sampled_points
#    there is no point in recalculating weights and location of points ...  
# 4) modify loss function to add info from channels and from batches
# 5) change radiuss into 3 dimensional array giving possibility of different radii for each axis
# 6) change the interpolation algorithm (and variance one) to get all voxels in the cube around the point and then calculate the value based on the distance to the point
#         take into account spacing of the current array

# Points from weights

In [ ]:
res=""

function save_point_on_line_for_lin(out_channel, x_add, y_add, z_add, weights_channel)
res=""
    # for i in 1:3

        curr="""
  control_points_out[x, y, z, $out_channel, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int((ceil(CUDA.blockIdx().z / num_blocks_z_pure)))] = (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add), Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(CUDA.blockIdx().z / num_blocks_z_pure))]
+
(sv_centers_mod[x+1, y+1, z+1, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(CUDA.blockIdx().z / num_blocks_z_pure))] - (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add), Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(CUDA.blockIdx().z / num_blocks_z_pure))])) * weights[x, y, z, ($weights_channel), Int(ceil(CUDA.blockIdx().z / num_blocks_z_pure))])

                                                                                                               
                                                                                                               """
        res=res*curr
                                                                                                            # end
return res
end


function save_point_for_oblique(out_channel, x_add, y_add, z_add, x_add_b, y_add_b, z_add_b, weights_channel)
    res=""
    # for i in 1:3
        curr="""

        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add)
        , Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))]
        +
        ((sv_centers_mod[x+($x_add_b), y+($y_add_b), z+($z_add_b), Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure))
        , Int(ceil(blockIdx().z  / num_blocks_z_pure))] - (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add)
        , Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))])) 
        * weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

                                                                                    """
        res=res*curr
    # end
    return res
end




"""
out_channel - channel to save the result in shared memory
x_add, y_add, z_add - coordinates of the point in the sv_centers
x_add_b, y_add_b, z_add_b - coordinates of the point in the main oblique in control points
weights_channel - channel of the weights
"""
function save_point_sv_center_to_oblique(out_channel, x_add, y_add, z_add, x_add_b, y_add_b, z_add_b, weights_channel)
    res=""
    # for i in 1:3
        curr="""

shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add)
, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))]
+
((control_points[x+($x_add_b), y+($y_add_b), z+($z_add_b), 4, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] 
 - (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add)
, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))])) 
* weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

                                                                                    """
        res=res*curr
    # end
    return res
end




"""
out_channel - channel to save the result in shared memory
x_add, y_add, z_add - coordinates of the point in the sv_centers
in_channel - channel in shared memory to which we want to refer
weights_channel - channel of the weights
"""
function save_point_sv_center_to_shared(out_channel, x_add, y_add, z_add, in_channel, weights_channel)

        return """

shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add)
, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))]
+
((shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel]
 - (sv_centers_mod[x+($x_add), y+($y_add), z+($z_add)
, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))])) 
* weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

"""

end



function save_point_for_oblique_to_oblique(out_channel, x_add, y_add, z_add, x_add_b, y_add_b, z_add_b, weights_channel)
    res=""
    # for i in 1:3
        curr="""

        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = 
        (control_points[x+($x_add), y+($y_add), z+($z_add), 4, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))]
                                                                                    +
((control_points[x+($x_add_b), y+($y_add_b), z+($z_add_b), 4, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] 
- (control_points[x+($x_add), y+($y_add), z+($z_add), 4, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))])) 
* weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

    """
        res=res*curr
    

# end

return res
end

"""
interpolate between two points save in control points
x_add, y_add, z_add - coordinates of the point in the first control point
x_add_b, y_add_b, z_add_b - coordinates of the point in the second control point
weights_channel - channel of the weights
cp1_channel - channel of the first control point
cp2_channel - channel of the second control point
out_channel - channel to save the result in shared memory
coord - coordinate 1 for x 2 for y and 3 for z on which we are currently working
"""
function save_point_for_cp_to_cp(out_channel, x_add, y_add, z_add, x_add_b, y_add_b, z_add_b, weights_channel, cp1_channel, cp2_channel,coord)
    return """

        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = 
        (control_points[x+($x_add), y+($y_add), z+($z_add), $cp1_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))]
                                                                                    +
    ((control_points[x+($x_add_b), y+($y_add_b), z+($z_add_b), $cp2_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))] 
    - (control_points[x+($x_add), y+($y_add), z+($z_add), $cp1_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))])) 
    * weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

        """

end


"""
interpolate between two points one in control points other in shared memory
x_add_b, y_add_b, z_add_b - coordinates of the point in the second control point
weights_channel - channel of the weights
in_channel_1 - channel of the first control point from shared memory
cp2_channel - channel of the second control point
out_channel - channel to save the result in shared memory
coord - coordinate 1 for x 2 for y and 3 for z on which we are currently working
"""
function save_point_for_shared_to_cp(out_channel, x_add_b, y_add_b, z_add_b, weights_channel, in_channel_1, cp2_channel,coord)
    return """

        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = 
        (shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_1]
                                                                                    +
    ((control_points[x+($x_add_b), y+($y_add_b), z+($z_add_b), $cp2_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))] 
    - (shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_1])) 
    * weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

        """

end



function get_point_between_shared(out_channel, in_channel_1, in_channel_2, weights_channel)

    res=""
    # for i in 1:3
        curr="""
        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = (shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_1]
        +
        (((shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_2] - (shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_1])) * weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))])))
    """
        res=res*curr
                                                                                # end

    return res
end




res="""
    if (x <= (cp_x) && y <= (cp_y) && z <= (cp_z) && x > 0 && y > 0 && z > 0)


        #lin_x
        $(save_point_on_line_for_lin(1, 0, 1, 1, 4))
        # #lin_y
        $(save_point_on_line_for_lin(2, 1, 0, 1, 5))
        #lin_z
        $(save_point_on_line_for_lin(3, 1, 1, 0, 6))
        #oblique main
        # save_point_for_oblique(1,x_add,y_add,z_add,x_add_b,y_add_b,z_add_b,7)
        #first we draw the line between 2 sv centers in x direction and get a point on it for each pair of sv centers that are neighbours in x
        #as we are moving in x we just need one weight for all of those points
            

        $(save_point_for_oblique(1, 0, 0, 0, 1, 0, 0, 7))
        $(save_point_for_oblique(2, 0, 1, 0, 1, 1, 0, 7))
        $(save_point_for_oblique(3, 0, 0, 1, 1, 0, 1, 7))
        $(save_point_for_oblique(4, 0, 1, 1, 1, 1, 1, 7))
        # #then we draw the line between 2 points calculated above centers in y direction and get a point on it for each pair of sv centers that are neighbours in y

        $(get_point_between_shared(1, 1, 2, 8))
        $(get_point_between_shared(3, 4, 4, 8))
        $(get_point_between_shared(1, 1, 3, 9))
        #saving output
        control_points_out[x, y, z, 4, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(CUDA.blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
    end

"""

filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/points_from_weights_unrolled.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


# now we need to 
# 1) print the code from above and use ctrl f step by step to check is it identical to already present unrolled Version
# 2) add support for batches - basically add blockid.y to each array at the end
# 3) add support for channels here we basically need separate unctions - kernels so we will simply sample variance  from tetr_dat or iterpolated value from out_sampled_points
#    there is no point in recalculating weights and location of points ...  
# 4) modify loss function to add info from channels and from batches
# 5) change radiuss into 3 dimensional array giving possibility of different radii for each axis
# 6) change the interpolation algorithm (and variance one) to get all voxels in the cube around the point and then calculate the value based on the distance to the point
#         take into account spacing of the current array

In [ ]:

# function generate_additional_oblique_code()

#     # ...existing code...
#     # Cell for generating the unrolled code for apply_weights_to_locs_kern_add_a (Revised)

#     # Assume the helper functions like save_point_sv_center_to_oblique, get_point_between_shared,
#     # save_point_sv_center_to_shared are defined in the notebook scope (e.g., cell 'a26314ce')
#     # and correctly handle the CUDA indexing (blockIdx, threadIdx, etc.).

#     # Define the number of additional points per side (pyramid iteration)
#     # Each iteration adds one point per pyramid per axis.
#     num_additional_oblique_points_per_side = 2 # Example value, change as needed

#     # --- Main Code Generation Logic ---
#     res = """
#         # CUDA Kernel Code for Calculating Additional Oblique Control Points
#         # This code is unrolled in Julia before kernel compilation.

#         # Check if the current thread is within the valid processing bounds
#         if (x <= (cp_x) && y <= (cp_y) && z <= (cp_z) && x > 0 && y > 0 && z > 0)

#     """

#     # Define the relative coordinates for the two main oblique points (apices of the pyramids)
#     # Oblique 1 is always at the origin relative to the current control point index (x,y,z)
#     oblique1_coords = (0, 0, 0)
#     # Oblique 2 depends on the axis being processed
#     oblique2_coords_list = [(1, 0, 0), (0, 1, 0), (0, 0, 1)] # For X, Y, Z axes respectively
#     axis_names = ["X", "Y", "Z"]

#     # Define the relative coordinates for the 4 Supervoxel (SV) centers forming the base of the pyramids.
#     # These are relative to the current control point index (x,y,z).
#     # Note: sv_centers_mod indices are shifted by +1 compared to control_points indices.
#     # hence 1,1,1 is really 0,0,0 so indicating the same point when we use indicies of control points out
#     # The coordinates depend on the axis being processed.
#     sv_centers_coords_per_axis = [
#         # X-axis specific SV centers relative to (x,y,z)
#         [(1, 1, 1), (1, 1, 0), (1, 0, 0), (1, 0, 1)],
#         # Y-axis specific SV centers relative to (x,y,z)
#         [(1, 1, 1), (1, 1, 0), (0, 1, 0), (0, 1, 1)],
#         # Z-axis specific SV centers relative to (x,y,z)
#         [(1, 1, 1), (1, 0, 1), (0, 0, 1), (0, 1, 1)]
#     ]


#     # Initialize counters for weights channels and output control point channels
#     weights_channel_counter = 10 # Start weights from channel 10
#     cp_out_num_counter = 5      # Start output control points from channel 5 (channels 1-4 are lin_x, lin_y, lin_z, oblique_main)

#     # Loop through each axis (X, Y, Z) to generate code for each set of additional oblique points
#     for axis_idx in 1:3
#         axis_name = axis_names[axis_idx]
#         oblique2_coords = oblique2_coords_list[axis_idx]
#         current_sv_centers_coords = sv_centers_coords_per_axis[axis_idx]

#         res *= """
#             # --------------------------------------------------------------------------
#             # --- Calculations for Additional Oblique Points - Axis $(axis_name) ---
#             # --------------------------------------------------------------------------
#     """

#         # --- Process First Pyramid (Apex: oblique1_coords @ (0,0,0)) ---
#         res *= """
#             # --- Axis $(axis_name) - Pyramid 1 (Apex: Main Oblique @ $(oblique1_coords)) ---
#             # Step a: Interpolate between the 4 relevant SV centers and the first main oblique point.
#             # Results are stored in shared memory channels 1-4.
#     """
#         # Step a (Unrolled): Interpolate from SV centers to the first main oblique
#         for i in 1:4
#             sv_coord = current_sv_centers_coords[i]
#             res *= "# Interpolating SV Center $(sv_coord) to Oblique 1 $(oblique1_coords)\n"
#             res *= save_point_sv_center_to_oblique(i, sv_coord[1], sv_coord[2], sv_coord[3], oblique1_coords[1], oblique1_coords[2], oblique1_coords[3], weights_channel_counter) * "\n"
#             weights_channel_counter += 1
#         end

#         # Iterate to create points within the first pyramid
#         for iter in 1:num_additional_oblique_points_per_side
#             res *= """
#             # --- Axis $(axis_name) - Pyramid 1, Iteration $(iter)/$(num_additional_oblique_points_per_side) ---
#             # Step b: Combine the 4 points stored in shared memory (channels 1-4) via weighted averages.
#             # Intermediate results in shared channels 5, 6. Final result for this iteration in channel 5.
#     """
#             # Step b (Unrolled): Combine points in shared memory
#             res *= "# Combine shared[1] and shared[2] -> shared[5]\n"
#             res *= get_point_between_shared(5, 1, 2, weights_channel_counter) * "\n"
#             weights_channel_counter += 1
#             res *= "# Combine shared[3] and shared[4] -> shared[6]\n"
#             res *= get_point_between_shared(6, 3, 4, weights_channel_counter) * "\n"
#             weights_channel_counter += 1
#             res *= "# Combine shared[5] and shared[6] -> shared[5] (final point for iteration)\n"
#             res *= get_point_between_shared(5, 5, 6, weights_channel_counter) * "\n"
#             weights_channel_counter += 1

#             # Step c (Unrolled): Save the resulting point to the output control points array
#             res *= """
#             # Step c: Save the calculated point to control_points_out channel $(cp_out_num_counter).
#             control_points_out[x, y, z, $(cp_out_num_counter), Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 5]
#     """
#             cp_out_num_counter += 1

#             # Step d (Unrolled): Prepare for the next iteration (if any) by interpolating
#             # between SV centers and the points calculated in Step a (still in shared channels 1-4).
#             # This overwrites shared memory channels 1-4.
#             if iter < num_additional_oblique_points_per_side
#                 res *= """
#             # Step d: Interpolate SV centers towards the intermediate points (from step a, stored in shared[1-4])
#             # This prepares shared memory channels 1-4 for the next iteration.
#     """
#                 for i in 1:4
#                     sv_coord = current_sv_centers_coords[i]
#                     res *= "# Interpolating SV Center $(sv_coord) towards shared[$(i)] -> shared[$(i)]\n"
#                     # Overwrite shared channels 1-4 with new intermediate points
#                     res *= save_point_sv_center_to_shared(i, sv_coord[1], sv_coord[2], sv_coord[3], i, weights_channel_counter) * "\n"
#                     weights_channel_counter += 1
#                 end
#             end # End if iter < num_additional_oblique_points_per_side
#         end # End loop for num_additional_oblique_points_per_side (Pyramid 1)

#         # --- Process Second Pyramid (Apex: oblique2_coords @ axis-dependent location) ---
#         res *= """

#             # --- Axis $(axis_name) - Pyramid 2 (Apex: Main Oblique @ $(oblique2_coords)) ---
#             # Step a: Interpolate between the 4 relevant SV centers and the second main oblique point.
#             # Results overwrite shared memory channels 1-4.
#     """
#         # Step a (Unrolled, repeated for Pyramid 2): Interpolate from SV centers to the second main oblique
#         for i in 1:4
#             sv_coord = current_sv_centers_coords[i]
#             res *= "# Interpolating SV Center $(sv_coord) to Oblique 2 $(oblique2_coords)\n"
#             res *= save_point_sv_center_to_oblique(i, sv_coord[1], sv_coord[2], sv_coord[3], oblique2_coords[1], oblique2_coords[2], oblique2_coords[3], weights_channel_counter) * "\n"
#             weights_channel_counter += 1
#         end

#         # Iterate to create points within the second pyramid
#         for iter in 1:num_additional_oblique_points_per_side
#             res *= """
#             # --- Axis $(axis_name) - Pyramid 2, Iteration $(iter)/$(num_additional_oblique_points_per_side) ---
#             # Step b: Combine the 4 points stored in shared memory (channels 1-4) via weighted averages.
#             # Intermediate results in shared channels 5, 6. Final result for this iteration in channel 5.
#     """
#             # Step b (Unrolled): Combine points in shared memory
#             res *= "# Combine shared[1] and shared[2] -> shared[5]\n"
#             res *= get_point_between_shared(5, 1, 2, weights_channel_counter) * "\n"
#             weights_channel_counter += 1
#             res *= "# Combine shared[3] and shared[4] -> shared[6]\n"
#             res *= get_point_between_shared(6, 3, 4, weights_channel_counter) * "\n"
#             weights_channel_counter += 1
#             res *= "# Combine shared[5] and shared[6] -> shared[5] (final point for iteration)\n"
#             res *= get_point_between_shared(5, 5, 6, weights_channel_counter) * "\n"
#             weights_channel_counter += 1

#             # Step c (Unrolled): Save the resulting point to the output control points array
#             res *= """
#             # Step c: Save the calculated point to control_points_out channel $(cp_out_num_counter).
#             control_points_out[x, y, z, $(cp_out_num_counter), Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 5]
#     """
#             cp_out_num_counter += 1

#             # Step d (Unrolled): Prepare for the next iteration (if any)
#             if iter < num_additional_oblique_points_per_side
#                 res *= """
#             # Step d: Interpolate SV centers towards the intermediate points (from step a, stored in shared[1-4])
#             # This prepares shared memory channels 1-4 for the next iteration.
#     """
#                 for i in 1:4
#                     sv_coord = current_sv_centers_coords[i]
#                     res *= "# Interpolating SV Center $(sv_coord) towards shared[$(i)] -> shared[$(i)]\n"
#                     # Overwrite shared channels 1-4 with new intermediate points
#                     res *= save_point_sv_center_to_shared(i, sv_coord[1], sv_coord[2], sv_coord[3], i, weights_channel_counter) * "\n"
#                     weights_channel_counter += 1
#                 end
#             end # End if iter < num_additional_oblique_points_per_side
#         end # End loop for num_additional_oblique_points_per_side (Pyramid 2)

#         res *= """
#             # --- End of Calculations for Axis $(axis_name) ---
#     """

#     end # End loop axis_idx

#     res *= """
#         end # End main if condition (x, y, z bounds check)
#     """

#     # Add total weights used for verification
#     # For each axis (3):
#     #   For each pyramid (2):
#     #     Step a: 4 weights
#     #     Iterations (num_additional_oblique_points_per_side):
#     #       Step b: 3 weights
#     #       Step d (if not last iter): 4 weights
#     total_weights_needed = ( (4) + num_additional_oblique_points_per_side * (3) + max(0, num_additional_oblique_points_per_side - 1) * (4) ) * 2 * 3
#     res *= "\n# Verification: Total weights needed starting from channel 10: $(total_weights_needed)"
#     # Add total control points generated
#     # For each axis (3):
#     #   For each pyramid (2):
#     #     Iterations (num_additional_oblique_points_per_side):
#     #       Step c: 1 control point
#     total_cp_generated = num_additional_oblique_points_per_side * 2 * 3
#     res *= "\n# Verification: Total control points generated (channels starting from 5): $(total_cp_generated)"
#     res *= "\n# Verification: Final weight channel used: $(weights_channel_counter - 1)"
#     res *= "\n# Verification: Final control point channel used: $(cp_out_num_counter - 1)"


#     # Save the generated code string to the specified file
#     filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/points_from_weights_additional_unrolled.txt"
#     open(filepath, "w") do file
#         write(file, res)
#     end

#     println("Generated unrolled code saved to: $(filepath)")
#     # You can also print `res` here if you want to see it in the notebook output
#     # println(res)
# end

# generate_additional_oblique_code()






res="""
    if (x <= (cp_x) && y <= (cp_y) && z <= (cp_z) && x > 0 && y > 0 && z > 0)
        # first between 2 neighbouring oblique control points in x,y or z direction depending on which additional oblique we are calculating
        # second and third between neighbouring sv centers in other axes (for oblique_x in y and z)
        # we get a point from step 2 and result is point between step 1 and current one
        
        #oblique x
        $(save_point_for_oblique(1, 1, 1, 1, 1, 1, 0, 10))
        $(save_point_for_oblique(2, 1, 1, 1, 1, 0, 1, 11))
        $(save_point_for_oblique_to_oblique(3, 0, 0, 0, 1, 0, 0, 12))
        $(get_point_between_shared(1, 1, 2, 13))
        $(get_point_between_shared(1, 1, 3, 14))

        control_points_out[x, y, z, 5, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]

        #oblique y
        $(save_point_for_oblique(1, 1, 1, 1, 0, 1, 1, 15))
        $(save_point_for_oblique(2, 1, 1, 1, 1, 1, 0, 16))
        $(save_point_for_oblique_to_oblique(3, 0, 0, 0, 0, 1, 0, 17))
        $(get_point_between_shared(1, 1, 2, 18))
        $(get_point_between_shared(1, 1, 3, 19))
        control_points_out[x, y, z, 6, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
        #oblique z
        $(save_point_for_oblique(1, 1, 1, 1, 0, 1, 1, 20))
        $(save_point_for_oblique(2, 1, 1, 1, 1, 0, 1, 21))
        $(save_point_for_oblique_to_oblique(3, 0, 0, 0, 0, 0, 1, 22))
        $(get_point_between_shared(1, 1, 2, 23))
        $(get_point_between_shared(1, 1, 3, 24))
        control_points_out[x, y, z, 7, Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure)), Int(ceil(blockIdx().z  / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]


    end

"""



filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/points_from_weights_additional_unrolled.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


In [ ]:
# Define Helper Functions

"""
interpolate between two points save in control points
x_add, y_add, z_add - coordinates of the point in the first control point
x_add_b, y_add_b, z_add_b - coordinates of the point in the second control point
weights_channel - channel of the weights
cp1_channel - channel of the first control point
cp2_channel - channel of the second control point
out_channel - channel to save the result in shared memory
coord - coordinate 1 for x 2 for y and 3 for z on which we are currently working
"""
function save_point_for_cp_to_cp(out_channel, x_add, y_add, z_add, x_add_b, y_add_b, z_add_b, weights_channel, cp1_channel, cp2_channel,coord)
    return """

        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = 
        (control_points[x+($x_add), y+($y_add), z+($z_add), $cp1_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))]
                                                                                    +
    ((control_points[x+($x_add_b), y+($y_add_b), z+($z_add_b), $cp2_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))] 
    - (control_points[x+($x_add), y+($y_add), z+($z_add), $cp1_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))])) 
    * weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

        """

end


"""
interpolate between two points one in control points other in shared memory
x_add_b, y_add_b, z_add_b - coordinates of the point in the second control point
weights_channel - channel of the weights
in_channel_1 - channel of the first control point from shared memory
cp2_channel - channel of the second control point
out_channel - channel to save the result in shared memory
coord - coordinate 1 for x 2 for y and 3 for z on which we are currently working
"""
function save_point_for_shared_to_cp(out_channel, x_add_b, y_add_b, z_add_b, weights_channel, in_channel_1, cp2_channel,coord)
    return """

        shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $out_channel] = 
        (shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_1]
                                                                                    +
    ((control_points[x+($x_add_b), y+($y_add_b), z+($z_add_b), $cp2_channel, $coord, Int(ceil(blockIdx().z  / num_blocks_z_pure))] 
    - (shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, $in_channel_1])) 
    * weights[x, y, z, ($weights_channel), Int(ceil(blockIdx().z  / num_blocks_z_pure))]))

        """

end

In [ ]:
# Initialize Parameters
num_additional_oblique_points_per_side = 2 # Example: 2 additional points per side
weight_index = 10 # Start weights from channel 10
cp_chan_out = 5   # Start output control points from channel 5
res = """
    # CUDA Kernel Code for Calculating Additional Oblique Control Points
    # This code is unrolled in Julia before kernel compilation.

    # Check if the current thread is within the valid processing bounds
    if (x <= (cp_x) && y <= (cp_y) && z <= (cp_z) && x > 0 && y > 0 && z > 0)

""" # Start the main result string

In [ ]:
# Generate Code for Oblique X Points
res_x = """
        # --- Calculations for Additional Oblique Points - Axis X ---
"""

# Calculate the first point for Oblique X
coord = "Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure))" # Coordinate for X axis
res_x *= """
        # Calculate first point for Oblique X (between lin_y and lin_z)
        $(save_point_for_cp_to_cp(1, 0, 0, 0, 0, 0, 0, weight_index, 2, 3, coord)) # cp1=lin_y(2), cp2=lin_z(3)
"""
weight_index += 1
res_x *= """
        # Save first point for Oblique X
        control_points_out[x, y, z, $cp_chan_out, $coord, Int(ceil(blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
"""
cp_chan_out += 1

# Loop for subsequent points for Oblique X
for i in 1:(num_additional_oblique_points_per_side - 1)
    res_x *= """
        # Calculate subsequent point $(i+1) for Oblique X (between shared[1] and oblique_main)
        $(save_point_for_shared_to_cp(1, 0, 0, 0, weight_index, 1, 4, coord)) # cp2=oblique_main(4)
"""
    weight_index += 1
    res_x *= """
        # Save subsequent point $(i+1) for Oblique X
        control_points_out[x, y, z, $cp_chan_out, $coord, Int(ceil(blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
"""
    cp_chan_out += 1
end

In [ ]:
# Generate Code for Oblique Y Points
res_y = """
        # --- Calculations for Additional Oblique Points - Axis Y ---
"""

# Calculate the first point for Oblique Y
coord = "Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure))" # Coordinate for Y axis (adjust if needed)
res_y *= """
        # Calculate first point for Oblique Y (between lin_z and lin_x)
        $(save_point_for_cp_to_cp(1, 0, 0, 0, 0, 0, 0, weight_index, 3, 1, coord)) # cp1=lin_z(3), cp2=lin_x(1)
"""
weight_index += 1
res_y *= """
        # Save first point for Oblique Y
        control_points_out[x, y, z, $cp_chan_out, $coord, Int(ceil(blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
"""
cp_chan_out += 1

# Loop for subsequent points for Oblique Y
for i in 1:(num_additional_oblique_points_per_side - 1)
    res_y *= """
        # Calculate subsequent point $(i+1) for Oblique Y (between shared[1] and oblique_main)
        $(save_point_for_shared_to_cp(1, 0, 0, 0, weight_index, 1, 4, coord)) # cp2=oblique_main(4)
"""
    weight_index += 1
    res_y *= """
        # Save subsequent point $(i+1) for Oblique Y
        control_points_out[x, y, z, $cp_chan_out, $coord, Int(ceil(blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
"""
    cp_chan_out += 1
end

In [ ]:
# Generate Code for Oblique Z Points
res_z = """
        # --- Calculations for Additional Oblique Points - Axis Z ---
"""

# Calculate the first point for Oblique Z
coord = "Int(ceil(CUDA.blockIdx().y / num_blocks_y_pure))" # Coordinate for Z axis (adjust if needed)
res_z *= """
        # Calculate first point for Oblique Z (between lin_x and lin_y)
        $(save_point_for_cp_to_cp(1, 0, 0, 0, 0, 0, 0, weight_index, 1, 2, coord)) # cp1=lin_x(1), cp2=lin_y(2)
"""
weight_index += 1
res_z *= """
        # Save first point for Oblique Z
        control_points_out[x, y, z, $cp_chan_out, $coord, Int(ceil(blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
"""
cp_chan_out += 1

# Loop for subsequent points for Oblique Z
for i in 1:(num_additional_oblique_points_per_side - 1)
    res_z *= """
        # Calculate subsequent point $(i+1) for Oblique Z (between shared[1] and oblique_main)
        $(save_point_for_shared_to_cp(1, 0, 0, 0, weight_index, 1, 4, coord)) # cp2=oblique_main(4)
"""
    weight_index += 1
    res_z *= """
        # Save subsequent point $(i+1) for Oblique Z
        control_points_out[x, y, z, $cp_chan_out, $coord, Int(ceil(blockIdx().z / num_blocks_z_pure))] = shared_arr[threadIdx().x, threadIdx().y, threadIdx().z, 1]
"""
    cp_chan_out += 1
end

In [ ]:
# Combine and Save Code
res *= res_x # Add Oblique X code
res *= res_y # Add Oblique Y code
res *= res_z # Add Oblique Z code

res *= """
    end # End main if condition (x, y, z bounds check)

    # Verification: Final weight index used: $(weight_index - 1)
    # Verification: Final control point channel used: $(cp_chan_out - 1)
"""

# Optionally save the generated code to a file
filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/points_from_weights_additional_unrolled_v2.txt"
open(filepath, "w") do file
    write(file, res)
end

println("Generated unrolled code saved to: $(filepath)")
# println(res) # Optionally print the generated code

# get weights

In [ ]:
  #on x dimension we are iterating over sv centers
  #threads on y are iterating on the same sv center the same parameter set and channel - divided just becouse of enzyme compilation issues
  #blocks on y encode direction by (( blockIdx().y%num_directions )+1) and which parameter set by (Int(ceil(blockIdx().y/num_directions)))
  # blocks z encode batch and channel

  num_indicies_per_block=7




  


res="""
if (threadIdx().x == 1 &  threadIdx().y == 1 )
"""

  for i in 1:3
          for j in 1:3
                  for k in 1:3
                          #Int(ceil(threadIdx().y/times_exec_per_dir)) indicates what parametrization we are using as we can use multiple threads in y dim to calculate the same direction
                          # and using the same parameter sets
                          res=res*""" 
                            shared_arr_kern[$i, $j, $k] = (conv_kernels[$i, $j, $k, Int(CUDA.blockIdx().z % channels_num)+1,(Int(ceil(blockIdx().y/num_directions)))
                            ,(( blockIdx().y%num_directions )+1) ])
                          """
                        end
          end
  end
  # @cuprintln("** $(shared_arr_kern[1,2,2,threadIdx().y]) **")  
res=res*"""
end
sync_threads()
"""


# point_index= (( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index)

dif_point_index_arr=[]
for point_index in 1:(num_indicies_per_block)

    # point_index= (((Int(floor((blockIdx().x/true_blocks_x)))*num_indicies_per_block)+1)+$point_index)

# we need to make sure that we are not going out of the array of directions indicies or that we try to work on its padding    

# res=res*"""
#  point_index=  (( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index)
#   if (point_index<=max_len_dir)
#     if (directions_indicies[(( blockIdx().y%num_directions )+1),point_index,1]>-1000)      
        
#       """

  #just a convolution - convolution weigts were loaded to the shared memory    
 loc_res_arr=[]     
for i in -1:1
    for j in -1:1
        for k in -1:1      
            loc_res="""(source_arr[ Int(floor((directions_indicies[(( blockIdx().y%num_directions )+1)
            ,(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1]+flat_sv_centers[(threadIdx().x + (blockIdx().x * CUDA.blockDim_x())),1])+($i)+beg_axis_pad))
            ,Int(floor(directions_indicies[(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),2]+flat_sv_centers[(threadIdx().x + (blockIdx().x * CUDA.blockDim_x())),2]+($j)+beg_axis_pad))
            ,Int(floor(directions_indicies[(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),3]+flat_sv_centers[(threadIdx().x + (blockIdx().x * CUDA.blockDim_x())),3]+($k)+beg_axis_pad))
            ,Int(CUDA.blockIdx().z %channels_num)+1,Int(ceil(CUDA.blockIdx().z / channels_num))  ] *shared_arr_kern[$i+2,$j+2,$k+2])
            """
            loc_res_arr=push!(loc_res_arr,loc_res)

        end
    end
  end
  
#   res=res*"""
#     conved=($(join(loc_res_arr, " + ")))
#     """






#   for  sh_i in 1:3
    #this is used to parametririze each entry so we will have some weighting and some parametrization that affects diffrently each voxel in the given direction
    #preserving information about direction
    res_loc_b="""(max(0,(($(join(loc_res_arr, " + ")))*(param_matrix_a[(Int(ceil(blockIdx().y/num_directions)))
    ,Int(CUDA.blockIdx().z %channels_num)+1
    ,(( blockIdx().y%num_directions )+1)
    ,(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,1])+
    param_matrix_a[(Int(ceil(blockIdx().y/num_directions)))
    ,Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1)
    ,(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,2] ))
    * (param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1
    ,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,3]))"""
#     res_loc_b="""
# (((max(0,((($(join(loc_res_arr, " + "))) #relu 
#         *(param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,1]))
#         +param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,2] )))#first and second parameter inside relu
#         )* (param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,3]))#third parameter outside relu
       
# #         """



dif_point_index_arr=push!(dif_point_index_arr,res_loc_b)

end
  res=res*"""

    shared_arr[threadIdx().x,threadIdx().y, 1]=(($(join(dif_point_index_arr, " + ")))/$num_indicies_per_block)
  sync_threads()

  """


# params_reduction

# now we need to aggregate information from all points in the given direction in a current block - we are aggregating just on y dimension of the block
to_add=[]
for  ii in [1,2,4,8,16,32,64,128,256,512]
  # loc_curr="Int(ceil(CUDA.blockDim_y()/($(ii)*2)))"
  # push!(to_add,loc_curr)
  res=res*"""
  ####$(ii)
    if ((threadIdx().y-1)%(2*$(ii))==0)
      if ((threadIdx().y+($(ii)))<=CUDA.blockDim_y())

        shared_arr[threadIdx().x,threadIdx().y,1]=((shared_arr[threadIdx().x,threadIdx().y,1]+shared_arr[threadIdx().x,threadIdx().y+($(ii)),1]))


      end
    end
  
  sync_threads()
  
"""
end  
res=res*"""   if (threadIdx().y==1)

    # for i in 2:CUDA.blockDim_y()
    #         shared_arr[threadIdx().x,1, 1]+=shared_arr[threadIdx().x,i, 1]
    #         shared_arr[threadIdx().x,1, 2]+=shared_arr[threadIdx().x,i, 2]
    #         shared_arr[threadIdx().x,1, 3]+=shared_arr[threadIdx().x,i, 3]
    # end

"""

#  out_summarised = zeros(Float32, batch_size, num_channels, size(flat_sv_centers, 1), num_shared_repr*num_params_exec, num_directions)

#we save aggregated information into global memory; we have all added on first thread in y dimension
    res=res*"""

out_summarised[Int(ceil(CUDA.blockIdx().z / channels_num)) 
,Int(CUDA.blockIdx().z %channels_num)+1
,(threadIdx().x + (blockIdx().x * CUDA.blockDim_x()))
, (1+(Int(ceil(blockIdx().y/num_directions)-1)))
, (( blockIdx().y%num_directions )+1) ]+=(shared_arr[threadIdx().x,1, 1]/ (voxel_counts[(( blockIdx().y%num_directions )+1)]/$num_indicies_per_block))
    
    """
res=res*"""
end
"""


  filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/get_weights.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)

# get weights b

In [ ]:
  #on x dimension we are iterating over sv centers
  #threads on y are iterating on the same sv center the same parameter set and channel - divided just becouse of enzyme compilation issues
  #blocks on y encode direction by (( blockIdx().y%num_directions )+1) and which parameter set by (Int(ceil(blockIdx().y/num_directions)))
  # blocks z encode batch and channel

res="""
if (threadIdx().x == 1 &  threadIdx().y == 1 )
"""

  for i in 1:3
          for j in 1:3
                  for k in 1:3
                          #Int(ceil(threadIdx().y/times_exec_per_dir)) indicates what parametrization we are using as we can use multiple threads in y dim to calculate the same direction
                          # and using the same parameter sets
                          res=res*""" 
                            shared_arr_kern[$i, $j, $k] = (conv_kernels[$i, $j, $k, Int(CUDA.blockIdx().z % channels_num)+1,(Int(ceil(blockIdx().y/num_directions)))
                            ,(( blockIdx().y%num_directions )+1) ])
                          """
                        end
          end
  end
  # @cuprintln("** $(shared_arr_kern[1,2,2,threadIdx().y]) **")  
res=res*"""
end
sync_threads()
"""


# point_index= (( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index)

dif_point_index_arr=[]
for point_index in 1:(num_indicies_per_block)

    # point_index= (((Int(floor((blockIdx().x/true_blocks_x)))*num_indicies_per_block)+1)+$point_index)

# we need to make sure that we are not going out of the array of directions indicies or that we try to work on its padding    

# res=res*"""
#  point_index=  (( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index)
#   if (point_index<=max_len_dir)
#     if (directions_indicies[(( blockIdx().y%num_directions )+1),point_index,1]>-1000)      
        
#       """

  #just a convolution - convolution weigts were loaded to the shared memory    
 loc_res_arr=[]     
for i in -1:1
    for j in -1:1
        for k in -1:1      
            loc_res="""(source_arr[ Int(floor((directions_indicies[(( blockIdx().y%num_directions )+1)
            ,(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1]+flat_sv_centers[(threadIdx().x + (blockIdx().x * CUDA.blockDim_x())),1])+($i)+beg_axis_pad))
            ,Int(floor(directions_indicies[(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),2]+flat_sv_centers[(threadIdx().x + (blockIdx().x * CUDA.blockDim_x())),2]+($j)+beg_axis_pad))
            ,Int(floor(directions_indicies[(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),3]+flat_sv_centers[(threadIdx().x + (blockIdx().x * CUDA.blockDim_x())),3]+($k)+beg_axis_pad))
            ,Int(CUDA.blockIdx().z %channels_num)+1,Int(ceil(CUDA.blockIdx().z / channels_num))  ] *shared_arr_kern[$i+2,$j+2,$k+2])
            """
            loc_res_arr=push!(loc_res_arr,loc_res)

        end
    end
  end
  
#   res=res*"""
#     conved=($(join(loc_res_arr, " + ")))
#     """






#   for  sh_i in 1:3
    #this is used to parametririze each entry so we will have some weighting and some parametrization that affects diffrently each voxel in the given direction
    #preserving information about direction
    res_loc_b="""(max(0,(($(join(loc_res_arr, " + ")))*(param_matrix_a[(Int(ceil(blockIdx().y/num_directions)))
    ,Int(CUDA.blockIdx().z %channels_num)+1
    ,(( blockIdx().y%num_directions )+1)
    ,(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,1])+
    param_matrix_a[(Int(ceil(blockIdx().y/num_directions)))
    ,Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1)
    ,(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,2] ))
    * (param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1
    ,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,3]))"""
#     res_loc_b="""
# (((max(0,((($(join(loc_res_arr, " + "))) #relu 
#         *(param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,1]))
#         +param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,2] )))#first and second parameter inside relu
#         )* (param_matrix_a[(Int(ceil(blockIdx().y/num_directions))),Int(CUDA.blockIdx().z %channels_num)+1,(( blockIdx().y%num_directions )+1),(( ((threadIdx().y)-1)*num_indicies_per_block)+$point_index),1,3]))#third parameter outside relu
       
# #         """



dif_point_index_arr=push!(dif_point_index_arr,res_loc_b)

end
  res=res*"""

    shared_arr[threadIdx().x,threadIdx().y, 1]=(($(join(dif_point_index_arr, " + ")))/$num_indicies_per_block)
  sync_threads()

  """


# params_reduction

# now we need to aggregate information from all points in the given direction in a current block - we are aggregating just on y dimension of the block
to_add=[]
for  ii in [1,2,4,8,16,32,64,128,256,512]
  # loc_curr="Int(ceil(CUDA.blockDim_y()/($(ii)*2)))"
  # push!(to_add,loc_curr)
  res=res*"""
  ####$(ii)
    if ((threadIdx().y-1)%(2*$(ii))==0)
      if ((threadIdx().y+($(ii)))<=CUDA.blockDim_y())
        shared_arr[threadIdx().x,threadIdx().y,1]=((shared_arr[threadIdx().x,threadIdx().y,1]+shared_arr[threadIdx().x,threadIdx().y+($(ii)),1]))
      end
    end
  
  sync_threads()
  
"""
end  
res=res*"""   if (threadIdx().y==1)

    # for i in 2:CUDA.blockDim_y()
    #         shared_arr[threadIdx().x,1, 1]+=shared_arr[threadIdx().x,i, 1]
    #         shared_arr[threadIdx().x,1, 2]+=shared_arr[threadIdx().x,i, 2]
    #         shared_arr[threadIdx().x,1, 3]+=shared_arr[threadIdx().x,i, 3]
    # end

"""

#  out_summarised = zeros(Float32, batch_size, num_channels, size(flat_sv_centers, 1), num_shared_repr*num_params_exec, num_directions)

#we save aggregated information into global memory; we have all added on first thread in y dimension
    res=res*"""

out_summarised[Int(ceil(CUDA.blockIdx().z / channels_num)) 
,Int(CUDA.blockIdx().z %channels_num)+1
,(threadIdx().x + (blockIdx().x * CUDA.blockDim_x()))
, (1+(Int(ceil(blockIdx().y/num_directions)-1)))
, (( blockIdx().y%num_directions )+1) ]+=(shared_arr[threadIdx().x,1, 1]/ (voxel_counts[(( blockIdx().y%num_directions )+1)]/$num_indicies_per_block))
    
    """
res=res*"""
end
"""


  filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/get_weights.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)

# mix_sv_info

In [ ]:
using Printf
mixing_radius=3






function save_unrolled_code(filename::String)
    open(filename, "w") do file
        @printf(file, "#mix_sv_info\n")
        for x_d in -mixing_radius:mixing_radius
            for y_d in -mixing_radius:mixing_radius
                for z_d in -mixing_radius:mixing_radius
                    @printf(file, """
if (blockIdx().x+%d) > 0 && (blockIdx().x+(%d)) <= cp_x && ((threadIdx().y + ((blockIdx().y - 1) * CUDA.blockDim_y()))+(%d)) > 0 && ((threadIdx().y + ((blockIdx().y - 1) * CUDA.blockDim_y()))+(%d)) <= cp_y && (z+(%d)) > 0 && (z+(%d)) <= cp_z
shared_arr[threadIdx().x, threadIdx().y] += (((max(
    ((Res[threadIdx().x, blockIdx().x+(%d), (threadIdx().y + ((blockIdx().y - 1) * CUDA.blockDim_y()))+(%d)
    , z+(%d), Int(ceil(CUDA.blockIdx().z / cp_z)) ]
    *mix_params[%d,%d,%d,threadIdx().x,2])
    +mix_params[%d,%d,%d,threadIdx().x,1])
    ,Res[ threadIdx().x, blockIdx().x+(%d), (threadIdx().y + ((blockIdx().y - 1) * CUDA.blockDim_y()))+(%d)
    , z+(%d), Int(ceil(CUDA.blockIdx().z / cp_z))]*0.000001))
    *mix_params[(%d),(%d),(%d),threadIdx().x,3]) )
end
                    """, x_d, x_d, y_d, y_d, z_d, z_d, x_d, y_d, z_d, x_d+mixing_radius+1, y_d+mixing_radius+1
                    , z_d+mixing_radius+1, x_d+mixing_radius+1, y_d+mixing_radius+1, z_d+mixing_radius+1
                    , x_d, y_d, z_d, x_d+mixing_radius+1, y_d+mixing_radius+1, z_d+mixing_radius+1)
                end
            end
        end
    end
end
filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/mix_weights.txt"

save_unrolled_code(filepath)

# get weights heavy 5x5x5

In [ ]:
# res="""
# using Pkg
# using ChainRulesCore, Zygote, CUDA, Enzyme
# # using CUDAKernels
# using KernelAbstractions
# # using KernelGradients
# using Zygote, Lux
# using Lux, Random
# import Optimisers, Plots, Random, Statistics, Zygote, LLVMLoopInfo
# using LinearAlgebra
# using Revise
# using Base.Threads


# ```
# first get all points around the sv center in this case for 5 voxels in each direction in 3 axes , then it 
#     uses 3x3x convolution to reduce number of points we are intrested in - so in practice after this convolution we can analize every second voxel
#    then we are using lrelu to mix information from all loaded convolved points 
#    last we are doing reductions - hevewer reductions are parametrirized and puprosfully performed multiple times in paralles so we will get 
#    5x5 result from 5x5x5 reduction
#    we are performing all operations separately for each parameter set, each channel and each batch
#    in order to make it a bit faster we will do 2 parameter sets at once in one kernel
# ```
# function kernel_analysis(conved, param_mixing
#         , param_reducing,param_reducing_b, param_reducing_c,result
#         ,num_blocks_y_true,l_relu_weight,dims,is_to_mix_weights,half_num_params)
        
#       # Calculate thread and block indices
#       # x = threadIdx().x
#       # y = threadIdx().y
#       # z = threadIdx().z
#       # bx = blockIdx().x
#       # by = blockIdx().y
#       # bz = blockIdx().z
#       # Calculate channel and parameter set index
      
#       # channel = UInt32(round((blockIdx().y % num_blocks_y_true)+1))
#       # param_set_idx = (UInt32(ceil((blockIdx().y / num_blocks_y_true)))+half_num_params*(((threadIdx().x) - 1) ÷ 5))

#         shared_mem = CuStaticSharedArray(Float32, (3,20,5,5))
#         shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=0.0
#          """






# function leaky_relu_in_shared_mem(l_relu_weight,channel,param_set_idx,param_tensor
#     ,c0,x0,y0,z0
#     ,c1,x1,y1,z1
#     ,c2,x2,y2,z2
#     ,xp,yp,zp
#     ,cpx,cpy,cpz)
#     return """
# shared_mem[3,$x0,$y0,$z0]=(shared_mem[$c1,$x1,$y1 ,$z1] * ($param_tensor)[1,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx])

# shared_mem[$c0,$x0,$y0,$z0]=(
#     max(  shared_mem[3,$x0,$y0,$z0]*($l_relu_weight)
#     ,(shared_mem[3,$x0,$y0,$z0]
#     +($param_tensor)[2,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx] )))

# shared_mem[3,$x0,$y0,$z0]=(shared_mem[$c2,$x2,$y2,$z2] * ($param_tensor)[3,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx])

# shared_mem[$c0,$x0,$y0,$z0]=((shared_mem[$c0,$x0,$y0,$z0]+    
#     max(shared_mem[3,$x0,$y0,$z0]*($l_relu_weight) ,shared_mem[3,$x0,$y0,$z0]
#     +($param_tensor)[4,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx] ))*(($param_tensor)[5,$cpx,$cpy,$cpz,$xp,$yp ,$zp
#     , $channel, $param_set_idx]/2 ))

    
#     """
# end    



# function waves_accum_shared_mem(l_relu_weight,channel,param_set_idx,param_tensor
#     ,c0,x0,y0,z0
#     ,c1,x1,y1,z1
#     ,c2,x2,y2,z2
#     ,xp,yp,zp
#     ,cpx,cpy,cpz)

# #shared_mem[3,$x0,$y0,$z0]=(shared_mem[$c1,$x1,$y1 ,$z1] * ($param_tensor)[1,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx])
# #shared_mem[3,$x0,$y0,$z0]=(shared_mem[$c2,$x2,$y2,$z2] * ($param_tensor)[3,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx]
    
# x=shared_mem[$c1,$x1,$y1 ,$z1]
# y=shared_mem[$c2,$x2,$y2,$z2]

#     return """



# shared_mem[$c0,$x0,$y0,$z0]=(
#       (shared_mem[$c1,$x1,$y1 ,$z1] + ($param_tensor)[2,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx])/ exp((shared_mem[$c1,$x1,$y1 ,$z1]*($param_tensor)[1,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx])
#      ))


# shared_mem[$c0,$x0,$y0,$z0]=((shared_mem[$c0,$x0,$y0,$z0]+    
#     max(shared_mem[3,$x0,$y0,$z0]*($l_relu_weight) ,shared_mem[3,$x0,$y0,$z0]
#     +($param_tensor)[4,$cpx,$cpy,$cpz,$xp,$yp ,$zp, $channel, $param_set_idx] ))*(($param_tensor)[5,$cpx,$cpy,$cpz,$xp,$yp ,$zp
#     , $channel, $param_set_idx]/2 ))

    
#     """


#     #     return """
# # shared_mem[3,$x0,$y0,$z0] = shared_mem[$c1,$x1,$y1,$z1] * ($param_tensor)[1,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx]

# # shared_mem[$c0,$x0,$y0,$z0] = shared_mem[3,$x0,$y0,$z0] * (1.0 / (1.0 + exp(-((
# #     ($param_tensor)[2,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx] * shared_mem[3,$x0,$y0,$z0] +
# #     ($param_tensor)[3,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx]))))

# # shared_mem[3,$x0,$y0,$z0] = shared_mem[$c2,$x2,$y2,$z2] * ($param_tensor)[4,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx]

# # shared_mem[$c0,$x0,$y0,$z0] += shared_mem[3,$x0,$y0,$z0] * (1.0 / (1.0 + exp(-((
# #     ($param_tensor)[5,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx] * shared_mem[3,$x0,$y0,$z0] +
# #     ($param_tensor)[6,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx]))))

# # shared_mem[$c0,$x0,$y0,$z0] *= ($param_tensor)[7,$cpx,$cpy,$cpz,$xp,$yp,$zp,$channel,$param_set_idx] / 2
# # """
# end



# # for i in 1:3
# #     for j in 1:3
# #         for k in 1:3
# #             res=res*"""
            
# #             shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z] += (source_arr[
# #                   flat_sv_centers[blockIdx().x, 1] + beg_axis_pad + (((mod(threadIdx().x - 1, 5) + 1)-3)*2)+($(i-2)) #ip
# #                 , flat_sv_centers[blockIdx().x, 2] + beg_axis_pad + ((threadIdx().y-3)*2)+($(j-2))#jp
# #                 , flat_sv_centers[blockIdx().x, 3] + beg_axis_pad + ((threadIdx().z-3)*2)+($(k-2))#kp
# #                 , UInt32(round((blockIdx().y % num_blocks_y_true)+1)), blockIdx().z] 
# #             * conv_kernels[$i,$j,$k,threadIdx().x,threadIdx().y,threadIdx().z
# #             , UInt32(round((blockIdx().y % num_blocks_y_true)+1)),UInt32(ceil((blockIdx().y / num_blocks_y_true)))])

# # """
# #         end
# #     end
# # end



# res=res*"""

#     shared_mem[2,threadIdx().x,threadIdx().y,threadIdx().z]=conved[blockIdx().x
#     ,(mod(threadIdx().x - 1, 5) + 1)
#     , threadIdx().y
#     , threadIdx().z
#     ,(UInt32(ceil((blockIdx().y / num_blocks_y_true)))+half_num_params*(((threadIdx().x) - 1) ÷ 5))
#     , UInt32(round((blockIdx().y % num_blocks_y_true)+1)), blockIdx().z] = shared_mem[threadIdx().x, threadIdx().y,threadIdx().z]
#     sync_threads()
#     # Step 3: Interaction in shared memory
# """


# # res=res*"""
# # if(is_to_mix_weights)

# # """
# for i in 1:5

# for j in 1:5
#     for k in 1:5
#         res=res*swish_in_shared_mem("l_relu_weight"
#         ,"UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#         ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_mixing"
#         ,"1","threadIdx().x","threadIdx().y","threadIdx().z"#output
#         ,"2","threadIdx().x","threadIdx().y","threadIdx().z"#input 1
#         ,"2","$i+((threadIdx().x - 1) ÷ 5 * 5)",j,k#input 2
#         ,"threadIdx().x","threadIdx().y","threadIdx().z","$i+((threadIdx().x - 1) ÷ 5 * 5)",j,k)#parameters
#     end
#     res=res*"""
#     sync_threads()
#     shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]/5
#     sync_threads()
#     """
# end  
# res=res*"""             
# sync_threads()              
# shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]/5
# sync_threads()
# """
# end
# # res=res*""" end

# # """





# res=res*"""


# # Synchronize threads
#     sync_threads()
#     #adding skip connection
#     #shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]+shared_mem[2,threadIdx().x,threadIdx().y,threadIdx().z]

#     shared_mem[2,threadIdx().x,threadIdx().y,threadIdx().z]=shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]
#     sync_threads()
#         # Step 4: reduction on x, y, and z axes 
#  if ((mod(threadIdx().x - 1, 5) + 1)==1)           
#         $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#         ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z" #output
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z" #input 1
#             ,"1","threadIdx().x+1","threadIdx().y","threadIdx().z" #input 2
#             ,"threadIdx().x+1","threadIdx().y","threadIdx().z","1","1","1" #parameters
#             ))          
        
#     end
#     if ((mod(threadIdx().x - 1, 5) + 1)==4)           
#         $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#         ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z" #output
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z" #input 1
#             ,"1","threadIdx().x+1","threadIdx().y","threadIdx().z" #input 2
#             ,"threadIdx().x+1","threadIdx().y","threadIdx().z","1","1","1"#parameters
#             ))         
        
#     end


        

#         if ((mod(threadIdx().x - 1, 5) + 1)==1 )           
#             #x3
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z"#output
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z"#input 1
#             ,"1","threadIdx().x+2","threadIdx().y","threadIdx().z"#input 2
#             ,"threadIdx().x+2","threadIdx().y","threadIdx().z"#parameters
#             ,"1","1","1" #parameters
#             )) 
#             #x4
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z"#output
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z"#input 1
#             ,"1","threadIdx().x+3","threadIdx().y","threadIdx().z"#input 2
#             ,"threadIdx().x+3","threadIdx().y","threadIdx().z"#parameters
#             ,"1","1","1" #parameters
#             ))        
#         end    
#         sync_threads()



#         #before we will get into y reduction let's populate all shared memory in x with accumulated values
#         shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=shared_mem[1,(mod(threadIdx().x - 1, 5)+1),threadIdx().y,threadIdx().z]/8
        
#         sync_threads()   
#         #y reduction
#         if (threadIdx().y==1)           
    
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_b"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z",
#             "1","threadIdx().x","2","threadIdx().z","threadIdx().x","1","threadIdx().z","1","1","1" #parameters
#             ))
#         end
#         if (threadIdx().y==4)           
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_b"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z"
#             ,"1","threadIdx().x","5","threadIdx().z","threadIdx().x","2","threadIdx().z","1","1","1" #parameters
#             ))
#         end
#         sync_threads()
#         #we have in y1 info from y1 and y2 and in y4 infor from y4 and y5
#         #so we just need to get info from y3 and y4 to y1
#         if (threadIdx().y==1 )           
#             #x3
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_b"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z #input 1"
#             ,"1","threadIdx().x","3","threadIdx().z","threadIdx().x","3","threadIdx().z","1","1","1" #parameters
#             ))
#             #x4
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_b"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z"
#             ,"1","threadIdx().x","4","threadIdx().z"
#             ,"threadIdx().x","4","threadIdx().z"
#             ,"1","1","1" #parameters
#             ))
#         end   
#         sync_threads()      
#         #we had accumulated all info on y dimension on y1 so we want to get it to all y
#         shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=shared_mem[1,threadIdx().x,1,threadIdx().z]/8
#         sync_threads()      

#         # z reducing
#         if (threadIdx().z==1 )          
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_c"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z"
#             ,"1","threadIdx().x","threadIdx().y","2"
#             ,"threadIdx().x","threadIdx().y","1"
#             ,"1","1","1" #parameters
#             ))
#         end
#         if (threadIdx().z==4 )          
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_c"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z"
#             ,"1","threadIdx().x","threadIdx().y","5","threadIdx().x","threadIdx().y","2","1","1","1" #parameters
#             ))
#         end
#         sync_threads()      
    
#         if (threadIdx().z==1 )          
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_c"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z"
#             ,"1","threadIdx().x","threadIdx().y","3","threadIdx().x","threadIdx().y","3","1","1","1" #parameters
#             ))
#             #x4
#             $(swish_in_shared_mem("l_relu_weight","UInt32(round((blockIdx().y % num_blocks_y_true)+1))"
#             ,"UInt32(ceil((blockIdx().y / num_blocks_y_true)))","param_reducing_c"
#             ,"1","threadIdx().x","threadIdx().y","threadIdx().z","1","threadIdx().x","threadIdx().y","threadIdx().z #input 1"
#             ,"1","threadIdx().x","threadIdx().y","4","threadIdx().x","threadIdx().y","4","1","1","1" #parameters
#             ) )
#         end    
#         sync_threads()      
#         # we had accumulated values in all axes so we can save it to global memory
#         # we had accumulated first x values and later we accumulated y and z block dim X times 
#         # so we avoided summarising information from threadblock into single number and now we have for example 5x5 values summarizing whole area 
#         # results are in all x and threads but z thread = 1 
    
    
#         # Step 5: Organize result
#         if (threadIdx().z==1 )       

#         result[mod(blockIdx().x - 1, dims[3]) + 1
#             ,div(mod(blockIdx().x  - 1, dims[2] * dims[3]), dims[3]) + 1
#             ,div(blockIdx().x  - 1, dims[2] * dims[3]) + 1 
#           ,(mod(threadIdx().x - 1, 5) + 1)
#           , threadIdx().y,(UInt32(ceil((blockIdx().y / num_blocks_y_true)))+half_num_params*(((threadIdx().x) - 1) ÷ 5))
#         , UInt32(round((blockIdx().y % num_blocks_y_true)+1)), blockIdx().z] = shared_mem[1,threadIdx().x, threadIdx().y,1]

#         end
#         return nothing
#     end


# """

# filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/get_weights_heavy5x5x5.txt"

# # Open the file in write mode
# file = open(filepath, "w")

# # Write the value of res to the file
# write(file, res)

# # Close the file
# close(file)

In [ ]:

res="""using Pkg
using ChainRulesCore, Zygote, CUDA, Enzyme
# using CUDAKernels
using KernelAbstractions
# using KernelGradients
using Zygote, Lux
using Lux, Random
import Optimisers, Plots, Random, Statistics, Zygote, LLVMLoopInfo
using LinearAlgebra
using Revise
using Base.Threads

function kernel_analysis_conv(source_arr, flat_sv_centers, conved, beg_axis_pad, conv_kernels
    ,num_blocks_y_true,half_num_params)
    
      # channel = UInt32(round((blockIdx().y % num_blocks_y_true)+1))
      # param_set_idx = (UInt32(ceil((blockIdx().y / num_blocks_y_true)))+half_num_params*(((threadIdx().x) - 1) ÷ 5))

    shared_mem = CuStaticSharedArray(Float32, (3,10,5,5))
    shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z]=flat_sv_centers[blockIdx().x, 1] + beg_axis_pad + (((mod(threadIdx().x - 1, 5) + 1)-3)*2)
    shared_mem[2,threadIdx().x,threadIdx().y,threadIdx().z]=flat_sv_centers[blockIdx().x, 2] + beg_axis_pad + ((threadIdx().y-3)*2)
    shared_mem[3,threadIdx().x,threadIdx().y,threadIdx().z]=flat_sv_centers[blockIdx().x, 3] + beg_axis_pad + ((threadIdx().z-3)*2)
    
    """


ttt=[]
for i in 1:3
    for j in 1:3
        for k in 1:3
            push!(ttt, """ (source_arr[
                 (shared_mem[1,threadIdx().x,threadIdx().y,threadIdx().z] +($(i-2))) #ip
                , (shared_mem[2,threadIdx().x,threadIdx().y,threadIdx().z]+($(j-2)))#jp
                , (shared_mem[3,threadIdx().x,threadIdx().y,threadIdx().z]+($(k-2)))#kp
                , UInt32(round((blockIdx().y % num_blocks_y_true)+1)), blockIdx().z] 
            * conv_kernels[$i,$j,$k,threadIdx().x,threadIdx().y,threadIdx().z
            , UInt32(round((blockIdx().y % num_blocks_y_true)+1))
            ,UInt32(ceil((blockIdx().y / num_blocks_y_true)))])
""")
        end
    end
end
res=res*""" conved[blockIdx().x
  ,(mod(threadIdx().x - 1, 5) + 1)
  , threadIdx().y
  , threadIdx().z
  ,(UInt32(ceil((blockIdx().y / num_blocks_y_true)))+half_num_params*(((threadIdx().x) - 1) ÷ 5))
, UInt32(round((blockIdx().y % num_blocks_y_true)+1)), blockIdx().z] =($(join(ttt, "+")))

return nothing

end
"""

filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/get_weights_heavy5x5x5_conv.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


# get_random_point_in_tetrs

In [ ]:
# get_random_point_in_tetrs
res=""

function intersect_lines_projected(line1_point1_x, line1_point1_y, line1_point1_z, 
    line1_point2_x, line1_point2_y, line1_point2_z, 
    line2_point1_x, line2_point1_y, line2_point1_z, 
    line2_point2_x, line2_point2_y, line2_point2_z, 
    out_shmem)
    return """
    # Calculate At * A
    if(threadIdx().y == 1)
    # AtA11
    shared_mem[threadIdx().x,1,1] = (($line1_point2_x) - ($line1_point1_x)) * (($line1_point2_x) - ($line1_point1_x)) +
    (($line1_point2_y) - ($line1_point1_y)) * (($line1_point2_y) - ($line1_point1_y)) +
    (($line1_point2_z) - ($line1_point1_z)) * (($line1_point2_z) - ($line1_point1_z))

    # AtA12
    shared_mem[threadIdx().x,2,1] = (($line1_point2_x) - ($line1_point1_x)) * -(($line2_point1_x) - ($line2_point2_x)) +
    (($line1_point2_y) - ($line1_point1_y)) * -(($line2_point1_y) - ($line2_point2_y)) +
    (($line1_point2_z) - ($line1_point1_z)) * -(($line2_point1_z) - ($line2_point2_z))
    end
    if(threadIdx().y == 2)
    # AtA21
    shared_mem[threadIdx().x,3,1] = -(($line2_point1_x) - ($line2_point2_x)) * (($line1_point2_x) - ($line1_point1_x)) +
    -(($line2_point1_y) - ($line2_point2_y)) * (($line1_point2_y) - ($line1_point1_y)) +
    -(($line2_point1_z) - ($line2_point2_z)) * (($line1_point2_z) - ($line1_point1_z))

    # AtA22
    shared_mem[threadIdx().x,1,2] = -(($line2_point1_x) - ($line2_point2_x)) * -(($line2_point1_x) - ($line2_point2_x)) +
    -(($line2_point1_y) - ($line2_point2_y)) * -(($line2_point1_y) - ($line2_point2_y)) +
    -(($line2_point1_z) - ($line2_point2_z)) * -(($line2_point1_z) - ($line2_point2_z))
    end
    # Calculate At * b
    if(threadIdx().y == 3)
    # Atb1
    shared_mem[threadIdx().x,2,2] = (($line1_point2_x) - ($line1_point1_x)) * (($line2_point2_x) - ($line1_point1_x)) +
    (($line1_point2_y) - ($line1_point1_y)) * (($line2_point2_y) - ($line1_point1_y)) +
    (($line1_point2_z) - ($line1_point1_z)) * (($line2_point2_z) - ($line1_point1_z))

    # Atb2
    shared_mem[threadIdx().x,3,2] = -(($line2_point1_x) - ($line2_point2_x)) * (($line2_point2_x) - ($line1_point1_x)) +
    -(($line2_point1_y) - ($line2_point2_y)) * (($line2_point2_y) - ($line1_point1_y)) +
    -(($line2_point1_z) - ($line2_point2_z)) * (($line2_point2_z) - ($line1_point1_z))
    end
    # Forward elimination
    sync_threads()
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,2,1] /= (shared_mem[threadIdx().x,1,1]+0.00000000000001)
    end
    if(threadIdx().y==2)
        shared_mem[threadIdx().x,2,2] /= (shared_mem[threadIdx().x,1,1]+0.00000000000001)
    end
    if(threadIdx().y==3)
        shared_mem[threadIdx().x,1,1] /= (shared_mem[threadIdx().x,1,1]+0.00000000000001)
    end
    sync_threads()
    
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,1,2] -= shared_mem[threadIdx().x,3,1] * shared_mem[threadIdx().x,2,1]
        shared_mem[threadIdx().x,3,2] -= shared_mem[threadIdx().x,3,1] * shared_mem[threadIdx().x,2,2]
        shared_mem[threadIdx().x,3,1] -= shared_mem[threadIdx().x,3,1] * shared_mem[threadIdx().x,1,1]
        
        # Normalize the second row
        shared_mem[threadIdx().x,3,2] /= (shared_mem[threadIdx().x,1,2]+0.00000000000001)
        shared_mem[threadIdx().x,1,2] /= (shared_mem[threadIdx().x,1,2]+0.00000000000001)
        
        # Back substitution
        shared_mem[threadIdx().x,1,2] = shared_mem[threadIdx().x,2,2] - shared_mem[threadIdx().x,2,1] * shared_mem[threadIdx().x,3,2]
    end
    sync_threads()
    
    sync_threads()

    # Calculate the intersection point
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,1, $out_shmem ] = (($line1_point1_x) + shared_mem[threadIdx().x,1,2] * (($line1_point2_x) - ($line1_point1_x)))
    end
    if(threadIdx().y==2)
        shared_mem[threadIdx().x,2, $out_shmem ] = (($line1_point1_y) + shared_mem[threadIdx().x,1,2] * (($line1_point2_y) - ($line1_point1_y)))
    end
    if(threadIdx().y==3)
        shared_mem[threadIdx().x,3, $out_shmem ] = (($line1_point1_z) + shared_mem[threadIdx().x,1,2] * (($line1_point2_z) - ($line1_point1_z)))
    end

    sync_threads()
    """
end




function is_point_on_segment(px, py, pz, x1, y1, z1, x2, y2, z2,out_s_1,out_s_2)
    return """

shared_mem[threadIdx().x,($out_s_1),($out_s_2)]= (0.5 * (1 + tanh(1000000 * ((($px) - ($x1)) * (($x2) - ($x1)) + (($py) - ($y1)) * (($y2) - ($y1)) 
    + (($pz) - ($z1)) * (($z2) - ($z1))) /    (((($x2) - ($x1))^2 + (($y2) - ($y1))^2 + (($z2) - ($z1))^2)+0.0000000001) 
    *((((($x2) - ($x1))^2 + (($y2) - ($y1))^2 + (($z2) - ($z1))^2) - 
    ((($px) - ($x1)) * (($x2) - ($x1)) + (($py) - ($y1)) * (($y2) - ($y1)) + (($pz) - ($z1)) * (($z2) - ($z1)))) 
    /     (((($x2) - ($x1))^2 + (($y2) - ($y1))^2 + (($z2) - ($z1))^2)+0.0000000001)) )))
    """
end

function intersect_and_clip(triangle_point1_x, triangle_point1_y, triangle_point1_z, triangle_point2_x, triangle_point2_y, triangle_point2_z)
    return """
        sync_threads()
        if(threadIdx().y == 1)
            $(is_point_on_segment(
                "shared_mem[threadIdx().x,1, 1]"," shared_mem[threadIdx().x,2, 1]"," shared_mem[threadIdx().x,3, 1]",
                "shared_mem[threadIdx().x,1, 3]", "shared_mem[threadIdx().x,2, 3]", "shared_mem[threadIdx().x,3, 3]",
                "shared_mem[threadIdx().x,1, 4]", "shared_mem[threadIdx().x,2, 4]"," shared_mem[threadIdx().x,3, 4]",
                "1", "2"
            ))
        end
        if(threadIdx().y == 2)
            $(is_point_on_segment(
                "shared_mem[threadIdx().x,1, 1]", "shared_mem[threadIdx().x,2, 1]", "shared_mem[threadIdx().x,3, 1]",
                triangle_point1_x, triangle_point1_y, triangle_point1_z,
                triangle_point2_x, triangle_point2_y, triangle_point2_z,
                2, 2
            ))
        end
        sync_threads()

        shared_mem[threadIdx().x,1, 2] = shared_mem[threadIdx().x,1, 2] * shared_mem[threadIdx().x,2, 2]

        sync_threads()
        shared_mem[threadIdx().x,threadIdx().y, 5] += shared_mem[threadIdx().x,threadIdx().y, 1] * shared_mem[threadIdx().x,1, 2]

        sync_threads()
    """
end






res=res*"""
function get_random_point_in_tetrs_kern(plan_tensor,weights,control_points_in,sv_centers_out,control_points_out,dims,max_index)
    
    #we need to check if we are not out of bounds
    if((threadIdx().x + ((blockIdx().x - 1) * CUDA.blockDim_x()))>max_index)
        return nothing
    end


    # shared_curr_plan=CuStaticSharedArray(Int16, (len_get_random_point_in_tetrs_kern,5,4))

    # if((threadIdx().x<21) && (threadIdx().y==1))
    #     plan_tensor[blockIdx().y, ((threadIdx().x%5)+1) ,Int(ceil(threadIdx().x/5))]=plan_tensor[blockIdx().y,,((threadIdx().x%5)+1),Int(ceil(threadIdx().x/5))]
    # end    
    # sync_threads()
    #calculating 3 dimensional index from linear 
    base_x=Int16(mod((threadIdx().x + ((blockIdx().x - 1) * CUDA.blockDim_x())) - 1, dims[1]) + 1)
    base_y=Int16(div(mod((threadIdx().x + ((blockIdx().x - 1) * CUDA.blockDim_x())) - 1, dims[2] * dims[1]), dims[1]) + 1)
    
    base_z=Int16(div((threadIdx().x + ((blockIdx().x - 1) * CUDA.blockDim_x())) - 1, dims[2] * dims[1]) + 1)
    # index=(threadIdx().x + ((blockIdx().x - 1) * CUDA.blockDim_x()))
    


    #ap_ap_base=intersect_line_plane(triangle_point1, triangle_point2, triangle_point3, apex1, apex2)
    # index = @index(Global)
    shared_mem=CuStaticSharedArray(Float32, (len_get_random_point_in_tetrs_kern,3,5))
    shared_mem[threadIdx().x,threadIdx().y,5]=0.0#just reset


    ########intersect_line_plane
    # saving normal
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,1,1]= (control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],2,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]) * (control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],3,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]) - (control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],3,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]) * (control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],2,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z])
    elseif(threadIdx().y==2)
        shared_mem[threadIdx().x,2,1]=(control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],3,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]) * (control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],1,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]) - (control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],1,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]) * (control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],3,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z])   
    elseif(threadIdx().y==3)
        shared_mem[threadIdx().x,3,1]=(control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],1,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]) * (control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],2,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]) - (control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],2,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]) * (control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],1,blockIdx().z] - control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z])
    end    
    # Synchronize threads
    sync_threads()

    if(threadIdx().y==1)
        shared_mem[threadIdx().x,1,2]=sqrt((shared_mem[threadIdx().x,1,1]^2)+(shared_mem[threadIdx().x,2,1]^2)+(shared_mem[threadIdx().x,3,1]^2))
    end
    sync_threads()
    shared_mem[threadIdx().x,threadIdx().y,1]=shared_mem[threadIdx().x,threadIdx().y,1]/(shared_mem[threadIdx().x,1,2]+0.00001)
    sync_threads()


    
    # dot_normal_line_dir is shared_mem[threadIdx().x,2,2] and dot_normal_diff is shared_mem[threadIdx().x,2,3]
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,2,2]=(shared_mem[threadIdx().x,1,1] * (sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              1,blockIdx().z] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              1,blockIdx().z]) + shared_mem[threadIdx().x,2,1] * (sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              2,blockIdx().z] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              2,blockIdx().z]) + shared_mem[threadIdx().x,3,1] * (sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              3,blockIdx().z] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              3,blockIdx().z]))
    end
    if(threadIdx().y==2)
        shared_mem[threadIdx().x,2,3] = shared_mem[threadIdx().x,1,1] * (control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              1,blockIdx().z]) + shared_mem[threadIdx().x,2,1] * (control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              2,blockIdx().z]) + shared_mem[threadIdx().x,3,1] * (control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              3,blockIdx().z])
    end


    sync_threads()
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,2,3]  = shared_mem[threadIdx().x,2,3] / (shared_mem[threadIdx().x,2,2]+0.0000001)
    end
    sync_threads()

    #saving ap_ap_base to the fourth column
    shared_mem[threadIdx().x,threadIdx().y,4]=sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
plan_tensor[blockIdx().y, 4, 2] + base_y,
plan_tensor[blockIdx().y, 4, 3] + base_z,
threadIdx().y] + shared_mem[threadIdx().x,2,3]  * (sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
plan_tensor[blockIdx().y, 5, 2] + base_y,
plan_tensor[blockIdx().y, 5, 3] + base_z,
threadIdx().y] - sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
plan_tensor[blockIdx().y, 4, 2] + base_y,
plan_tensor[blockIdx().y, 4, 3] + base_z,
threadIdx().y]) 

    ##################################get_random_point_in_triangle
    #base_point=get_random_point_in_triangle(triangle_point1, triangle_point2, triangle_point3,weights[1],weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  2 ,blockIdx().z],weights[3])
    #we get a random point on a base triangle that is shared between two tetrahedrons
    
    
    
    shared_mem[threadIdx().x,threadIdx().y,3] = (1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  3 
    ,blockIdx().z]) * ((1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  1 ,blockIdx().z]) * control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],threadIdx().y] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  1 ,blockIdx().z] * control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],threadIdx().y]) +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  3 ,blockIdx().z] * ((1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  2 ,blockIdx().z]) * control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],threadIdx().y] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  2 ,blockIdx().z] * control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],threadIdx().y])
    sync_threads()


    ############# ###########################
            #distance_to_triangle_edge(triangle_point1, triangle_point2, triangle_point3, P0, P1) P0 is base_point P1 so [threadIdx().y,3] is ap_ap_base so [threadIdx().y,4]

    ##############################################
    ### intersect_and_clip is a macro that saves and acumulates the result in shared_mem[threadIdx().x,y,5]
   
    # shared_mem[threadIdx().x,1,3] shared_mem[threadIdx().x,1,4]
    $(intersect_lines_projected("control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]"
    , "control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],3,blockIdx().z]"
    , "shared_mem[threadIdx().x,1,3]","shared_mem[threadIdx().x,2,3]","shared_mem[threadIdx().x,3,3]"
    , "shared_mem[threadIdx().x,1,4]","shared_mem[threadIdx().x,2,4]","shared_mem[threadIdx().x,3,4]","1") ) #save into shared_mem[threadIdx().x,y,1]
    sync_threads()
    $(intersect_and_clip("control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]"," control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],3,blockIdx().z]"))

    $(intersect_lines_projected("control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],3,blockIdx().z]"
    , "control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],3,blockIdx().z]"
    , "shared_mem[threadIdx().x,1,3]","shared_mem[threadIdx().x,2,3]","shared_mem[threadIdx().x,3,3]"
    , "shared_mem[threadIdx().x,1,4]","shared_mem[threadIdx().x,2,4]","shared_mem[threadIdx().x,3,4]",1)) #save into shared_mem[threadIdx().x,y,1]
    sync_threads()
    $(intersect_and_clip("control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],3,blockIdx().z]"," control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                            plan_tensor[blockIdx().y, 2, 2] + base_y,
                                            plan_tensor[blockIdx().y, 2, 3] + base_z,
                                            plan_tensor[blockIdx().y, 2, 4],3,blockIdx().z]"))

    $(intersect_lines_projected("control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]"
    , "control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],3,blockIdx().z]"
    , "shared_mem[threadIdx().x,1,3]","shared_mem[threadIdx().x,2,3]","shared_mem[threadIdx().x,3,3]"
    , "shared_mem[threadIdx().x,1,4]","shared_mem[threadIdx().x,2,4]","shared_mem[threadIdx().x,3,4]","1") ) #save into shared_mem[threadIdx().x,y,1]
    sync_threads()
    $(intersect_and_clip("control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                            plan_tensor[blockIdx().y, 1, 2] + base_y,
                                            plan_tensor[blockIdx().y, 1, 3] + base_z,
                                            plan_tensor[blockIdx().y, 1, 4],3,blockIdx().z]"," control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],1,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],2,blockIdx().z]","control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                                            plan_tensor[blockIdx().y, 3, 2] + base_y,
                                            plan_tensor[blockIdx().y, 3, 3] + base_z,
                                            plan_tensor[blockIdx().y, 3, 4],3,blockIdx().z]"))

    #now we have accumulated information about p_edge in shared_mem[threadIdx().x,1,5]

    #get d_p_edge in shared_mem[threadIdx().x,1,2] and norm(section) in shared_mem[threadIdx().x,1,1]
    shared_mem[threadIdx().x,threadIdx().y,2]=(shared_mem[threadIdx().x,threadIdx().y,5]-shared_mem[threadIdx().x,threadIdx().y,3])^2
    shared_mem[threadIdx().x,threadIdx().y,1]=(shared_mem[threadIdx().x,threadIdx().y,4]-shared_mem[threadIdx().x,threadIdx().y,3])^2

    sync_threads()
    if(threadIdx().y==1)
        shared_mem[threadIdx().x,1,2]=sqrt(shared_mem[threadIdx().x,1,2]+shared_mem[threadIdx().x,2,2]+shared_mem[threadIdx().x,3,2])
    end
    if(threadIdx().y==2)
        shared_mem[threadIdx().x,1,1]=sqrt(shared_mem[threadIdx().x,1,1]+shared_mem[threadIdx().x,2,1]+shared_mem[threadIdx().x,3,1])
    end
    sync_threads()
    #d_p_edge in shared_mem[threadIdx().x,1,2] and norm(section) in shared_mem[threadIdx().x,1,1] so we need to get the minimum of them and multiply by normalize (shared_mem[threadIdx().x,4]-shared_mem[threadIdx().x,3] so apapbase - base_point)
    # and add it to point from the base we will save it on the ap ap base place in shared memory as it is not needed anymore
    #we get edge_point_ap_common in shared_mem[threadIdx().x,y,4]
    shared_mem[threadIdx().x,threadIdx().y,4] =shared_mem[threadIdx().x,threadIdx().y,3]+(min(shared_mem[threadIdx().x,1,2],shared_mem[threadIdx().x,1,1])*((shared_mem[threadIdx().x,threadIdx().y,4]-shared_mem[threadIdx().x,threadIdx().y,3])/(shared_mem[threadIdx().x,1,1]+0.000001)))
    # sync_threads()
    sync_threads()
    #we get edge_point_ap2 in shared_mem[threadIdx().x,y,5]
    $(intersect_lines_projected("sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              1,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              2,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              3,blockIdx().z]"
    , "shared_mem[threadIdx().x,1,3]","shared_mem[threadIdx().x,2,3]","shared_mem[threadIdx().x,3,3]"
    , "shared_mem[threadIdx().x,1,4]","shared_mem[threadIdx().x,2,4]","shared_mem[threadIdx().x,3,4]"
    ,"sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              1,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              2,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              3,blockIdx().z]"
    ,"5"))
    #we get edge_point_ap1 in shared_mem[threadIdx().x,y,1] we do both on the same thread as shared mem column 1 and 2 is used internally in the macro    
    $(intersect_lines_projected("sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              1,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              2,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                              plan_tensor[blockIdx().y, 5, 2] + base_y,
                              plan_tensor[blockIdx().y, 5, 3] + base_z,
                              3,blockIdx().z]"
    , "shared_mem[threadIdx().x,1,3]","shared_mem[threadIdx().x,2,3]","shared_mem[threadIdx().x,3,3]"
    , "shared_mem[threadIdx().x,1,4]","shared_mem[threadIdx().x,2,4]","shared_mem[threadIdx().x,3,4]"
    ,"sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              1,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              2,blockIdx().z]","sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                              plan_tensor[blockIdx().y, 4, 2] + base_y,
                              plan_tensor[blockIdx().y, 4, 3] + base_z,
                              3,blockIdx().z]"
    ,"1"))
       
    sync_threads()  

    # #p1
    shared_mem[threadIdx().x,threadIdx().y,1]=((1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  4 ,blockIdx().z]) * shared_mem[threadIdx().x,threadIdx().y,3] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  4 ,blockIdx().z] * shared_mem[threadIdx().x,threadIdx().y,1])
    # #p2
    shared_mem[threadIdx().x,threadIdx().y,3]=((1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  5 ,blockIdx().z]) * shared_mem[threadIdx().x,threadIdx().y,4] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  5 ,blockIdx().z] * shared_mem[threadIdx().x,threadIdx().y,3])

    #using weights we get the final point
    # res[threadIdx().y]=(1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  6 ,blockIdx().z]) * ((1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  4 ,blockIdx().z]) * shared_mem[threadIdx().x,threadIdx().y,3] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  4 ,blockIdx().z] * shared_mem[threadIdx().x,threadIdx().y,1]) +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  6 ,blockIdx().z] * ((1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  5 ,blockIdx().z]) * shared_mem[threadIdx().x,threadIdx().y,4] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  4 ,blockIdx().z] * shared_mem[threadIdx().x,threadIdx().y,3])
    
    
#     #x = (1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  6 ,blockIdx().z]) * shared_mem[threadIdx().x,threadIdx().y,1] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  6 ,blockIdx().z] * shared_mem[threadIdx().x,threadIdx().y,3]
    


#clamping the value so it would not be futher in any direction than futherest point (rare pathological cases can happen)

shared_mem[threadIdx().x,threadIdx().y,3]=(1 -weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  6 ,blockIdx().z]) * shared_mem[threadIdx().x,threadIdx().y,1] +weights[base_x,base_y,base_z,((blockIdx().y-1)*6)+24+  6 ,blockIdx().z] * shared_mem[threadIdx().x,threadIdx().y,3]

# minimums in shared 1 maximums at shared 2
shared_mem[threadIdx().x,threadIdx().y,1]=min(
    min(
        min(
            min(
                control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                  plan_tensor[blockIdx().y, 1, 2] + base_y,
                                  plan_tensor[blockIdx().y, 1, 3] + base_z,
                                  plan_tensor[blockIdx().y, 1, 4], threadIdx().y, blockIdx().z],
                control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                  plan_tensor[blockIdx().y, 2, 2] + base_y,
                                  plan_tensor[blockIdx().y, 2, 3] + base_z,
                                  plan_tensor[blockIdx().y, 2, 4], threadIdx().y, blockIdx().z]
            ),
            control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                              plan_tensor[blockIdx().y, 3, 2] + base_y,
                              plan_tensor[blockIdx().y, 3, 3] + base_z,
                              plan_tensor[blockIdx().y, 3, 4], threadIdx().y, blockIdx().z]
        ),
        sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                       plan_tensor[blockIdx().y, 4, 2] + base_y,
                       plan_tensor[blockIdx().y, 4, 3] + base_z,
                       threadIdx().y, blockIdx().z]
    ),
    sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                   plan_tensor[blockIdx().y, 5, 2] + base_y,
                   plan_tensor[blockIdx().y, 5, 3] + base_z,
                   threadIdx().y, blockIdx().z]
)

shared_mem[threadIdx().x,threadIdx().y,2]=max(
    max(
        max(
            max(
                control_points_in[plan_tensor[blockIdx().y, 1, 1] + base_x,
                                  plan_tensor[blockIdx().y, 1, 2] + base_y,
                                  plan_tensor[blockIdx().y, 1, 3] + base_z,
                                  plan_tensor[blockIdx().y, 1, 4], threadIdx().y, blockIdx().z],
                control_points_in[plan_tensor[blockIdx().y, 2, 1] + base_x,
                                  plan_tensor[blockIdx().y, 2, 2] + base_y,
                                  plan_tensor[blockIdx().y, 2, 3] + base_z,
                                  plan_tensor[blockIdx().y, 2, 4], threadIdx().y, blockIdx().z]
            ),
            control_points_in[plan_tensor[blockIdx().y, 3, 1] + base_x,
                              plan_tensor[blockIdx().y, 3, 2] + base_y,
                              plan_tensor[blockIdx().y, 3, 3] + base_z,
                              plan_tensor[blockIdx().y, 3, 4], threadIdx().y, blockIdx().z]
        ),
        sv_centers_out[plan_tensor[blockIdx().y, 4, 1] + base_x,
                       plan_tensor[blockIdx().y, 4, 2] + base_y,
                       plan_tensor[blockIdx().y, 4, 3] + base_z,
                       threadIdx().y, blockIdx().z]
    ),
    sv_centers_out[plan_tensor[blockIdx().y, 5, 1] + base_x,
                   plan_tensor[blockIdx().y, 5, 2] + base_y,
                   plan_tensor[blockIdx().y, 5, 3] + base_z,
                   threadIdx().y, blockIdx().z]
)

shared_mem[threadIdx().x, threadIdx().y, 3] = max(
    shared_mem[threadIdx().x, threadIdx().y, 1],
    min(
        shared_mem[threadIdx().x, threadIdx().y, 3],
        shared_mem[threadIdx().x, threadIdx().y, 2]
    )
)

control_points_out[base_x,base_y,base_z,blockIdx().y+7,threadIdx().y ,blockIdx().z]=shared_mem[threadIdx().x,threadIdx().y,3]


    return nothing
end    
"""






res=res*"""
end
"""
filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/get_points_per_triangle.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


# sinusoid loss

In [ ]:

num_sinusoids_per_bank=4
num_texture_banks=8
# num_sinusoids_per_bank=4
# num_texture_banks=30

res="""
"""

to_add=[]
for t in 1 : num_texture_banks
    for sin_i in 1:num_sinusoids_per_bank

        # x -out_sampled_points[blockIdx().y, index, 3, blockIdx().z]
        # y -out_sampled_points[blockIdx().y, index, 4, blockIdx().z]
        # z -out_sampled_points[blockIdx().y, index, 5, blockIdx().z]
        #p=sin_p[index,sin_i,:,blockIdx().z]
        # alpha=p[1]*2*π
        # beta=p[2]*2*π
        # gamma=p[3]*2*π
        # wavelength=p[4]*max_wavelength
        # offset_x=p[5]*max_wavelength
        # offset_y=p[6]*max_wavelength
        # offset_z=p[7]*max_wavelength
        # base=p[8] *max_amplitude
        # amplitude=p[9]*max_amplitude

        # sin(2 * π / p[4] * ((x+p[5]) * cos(p[1]) + (y+p[6]) * cos(p[2]) + (z+p[7]) * cos(p[3])))+p[8]
        
        # sin(2 * π / p[4] * ((x+p[5]) * cos(p[1]) + (y+p[6]) * cos(p[2]) + (z+p[7]) * cos(p[3])))+p[8]
        #each weight is between 0 and 1 becouse sigmoid is last layer in layers to get sinusoid weights
        #so in case of all angls we multiply it by 2π
        #in case of offsets and wavelenths we multiply by max_wavelength


        push!(to_add,"""(((sin(2 * π / (texture_bank_p[$(t)+texture_bank_begin_index,$(sin_i),4]*max_wavelength)#wavelength
                * ((shared_arr[threadIdx().x,1])
                * cos(texture_bank_p[$(t)+texture_bank_begin_index,$(sin_i),1]*2*π)
                + (shared_arr[threadIdx().x,2]) 
                * cos(texture_bank_p[$(t)+texture_bank_begin_index,$(sin_i),2]*2*π) 
                + (shared_arr[threadIdx().x,3]) 
                * cos(texture_bank_p[$(t)+texture_bank_begin_index,$(sin_i),3]*2*π)))
                *(texture_bank_p[$(t)+texture_bank_begin_index,$(sin_i),5]*max_amplitude )* ((shared_arr_b[5]*2)+0.5)  )
                +(shared_arr_b[4] *max_amplitude)-min_value)*(sin_p[blockIdx().y, $(t)+texture_bank_begin_index+5 ,blockIdx().z]   ))""")
        # res=res*"""
        # shared_arr[threadIdx().x]+=(((sin(2 * π / (texture_bank_p[$(t),$(sin_i),4]*max_wavelength)#wavelength
        #  * ((out_sampled_points[blockIdx().y, index, 3, blockIdx().z]+(shared_arr_b[1]*max_wavelength))
        #   * cos(texture_bank_p[$(t),$(sin_i),1]*2*π)
        #    + (out_sampled_points[blockIdx().y, index, 4, blockIdx().z]+shared_arr_b[2]*max_wavelength) 
        #    * cos(texture_bank_p[$(t),$(sin_i),2]*2*π) 
        #    + (out_sampled_points[blockIdx().y, index, 5, blockIdx().z]+shared_arr_b[3]*max_wavelength) 
        #    * cos(texture_bank_p[$(t),$(sin_i),3]*2*π)))
        #    *(texture_bank_p[$(t),$(sin_i),5]*max_amplitude )* ((shared_arr_b[5]*2)+0.5)  )
        #    +(shared_arr_b[4] *max_amplitude)-min_value)*(sin_p[blockIdx().y, $(t)+5 ,blockIdx().z]   ))

        # """


    end    
end

res=res*join(to_add,"+")



filepath = "/workspaces/superVoxelJuliaCode_lin_sampl/superVoxelJuliaCode/data/sinusoid_loss.txt"

# Open the file in write mode
file = open(filepath, "w")

# Write the value of res to the file
write(file, res)

# Close the file
close(file)


In [ ]:
# shared_arr[threadIdx().x]+=(((sin(2 * π / (texture_bank_p[$(t),$(sin_i),4]*max_wavelength)#wavelength
# * ((shared_arr[threadIdx().x,1])
#  * cos(texture_bank_p[$(t),$(sin_i),1]*2*π)
#   + (shared_arr[threadIdx().x,2]) 
#   * cos(texture_bank_p[$(t),$(sin_i),2]*2*π) 
#   + (shared_arr[threadIdx().x,3]) 
#   * cos(texture_bank_p[$(t),$(sin_i),3]*2*π)))
#   *(texture_bank_p[$(t),$(sin_i),5]*max_amplitude )* ((shared_arr_b[5]*2)+0.5)  )
#   +(shared_arr_b[4] *max_amplitude)-min_value)*(sin_p[blockIdx().y, $(t)+5 ,blockIdx().z]   ))

aa=[1,2,3,45,6,7,8,9,10]
indd=800
vall=aa[min(size(aa)[1],max(1,indd))]
vall
